In [1]:
import os
import numpy as np
import zipfile
import io
import inspect
import cv2
from sklearn.metrics.pairwise import cosine_distances
from pathlib import Path

from openvino.inference_engine import IECore
from openvino.runtime import Core

In [2]:
core = Core()

In [3]:
devices = core.available_devices

for device in devices:
    device_name = core.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

CPU: Intel(R) Core(TM) i5-9600K CPU @ 3.70GHz
GPU: Intel(R) UHD Graphics 630 (iGPU)


In [4]:
# models = os.path.join('models', os.listdir('models')[1])

In [5]:
classification_model_xml = 'image-retrieval-0001/FP32/image-retrieval-0001.xml'
# classification_model_xml = 'models/vehicle-detection-0200/FP32/vehicle-detection-0200.xml'

model = core.read_model(model=classification_model_xml)
compiled_model = core.compile_model(model=model, device_name="CPU")

In [6]:
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

In [7]:
print(f"input precision: {input_layer.element_type}")
print(f"input shape: {input_layer.shape}")

input precision: <Type: 'float32'>
input shape: [1,224,224,3]


In [8]:
def preprocess_image(image_path, h, w):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (w, h))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)  # Convert to float32
#     image = image.transpose((0, 3, 1, 2))  # Interchange dimensions to [1, 3, 224, 224]
    image = np.array(image)
#     print(image)
    blob = np.expand_dims(image, axis=0)  # Add batch dimension
#     print(blob.shape)
#     blob = blob.transpose((0, 3, 1, 2))
    return blob
    

In [9]:
query_image_path = os.path.join('images', os.listdir('images')[0]) # возвращаем путь к картинке по которой ищем совпадение
# preprocessed_query_image = preprocess_image(query_image_path, 224, 224)
preprocessed_query_image = preprocess_image(query_image_path, 224, 224)

In [10]:
result_infer = compiled_model([preprocessed_query_image])[output_layer] # обрабатываем картинку
result_index = np.argmax(result_infer)

In [11]:
# Preprocess the batch of images
batch_images_path = 'archive/ds_test.zip'
batch_images_dir = 'unpack_archive'

with zipfile.ZipFile(batch_images_path, 'r') as zip_ref:
    zip_ref.extractall(batch_images_dir)
    zip_ref.close()

batch_images = []
for image_file in os.listdir(batch_images_dir):
    image_path = os.path.join(batch_images_dir, image_file)
    preprocessed_image = preprocess_image(image_path, 224, 224)
    batch_images.append(preprocessed_image)

batch_images = np.array(batch_images)

result_batch = []
for i in range(len(batch_images)):
    result = compiled_model([batch_images[i]])[output_layer]
    result_batch.append(result)
    
result_batch = np.array(result_batch)

In [12]:
index_box = []
for i in range(len(result_batch)):
    index = np.argmax(result_batch[i])
    index_box.append(index)

In [13]:
comparation = {}

In [14]:
for i in range(len(result_batch)):
    similarities = 1 - cosine_distances(result_infer, result_batch[i])
    percentages = similarities * 100
    comparation[os.listdir(batch_images_dir)[i]] = percentages[0][0]

In [15]:
best_similarity = max(comparation, key= lambda x: comparation[x]) # находим максимальное значение
print(f"Best similarity is : {best_similarity}", comparation[best_similarity], '%') # функция .item() извлекает данные из nd.array

Best similarity is : 7HWbnKehTfCIM8TuWrwnfA_375x500.jpg 82.63838 %


In [16]:
good_list = []

In [17]:
for k, v in comparation.items():
    if v > 50:
        good_list.append(k)
        good_list.append(v)

In [18]:
good_list

['5RcAmLXUQAOEe4z_oK93rg_pl_480x640.png',
 68.21366,
 '6dZ44MdaQvSrGM-rgMvzMg_375x500.jpg',
 50.61029,
 '6jqMPunZS0qc7DJIXYFk7A_pl_480x640.png',
 55.519115,
 '6nDmGZd0Tk2AmZgWVWDEBQ_375x500.jpg',
 67.79724,
 '6Nqq2cxdTnOkAuwqF9QqVg_375x500.jpg',
 52.501072,
 '6tYKYkW3SR6kmCsK4Egbpg_375x500.jpg',
 59.448772,
 '6U0mVoVPTZWJxl9Yp--fvw_375x500.jpg',
 55.036205,
 '72NNZMlcTo6pgCrY9lWBEQ_375x500.jpg',
 61.7308,
 '7BTILrnMRVm5-GTQOW56jQ_375x500.jpg',
 50.76937,
 '7dZ3Cg0VSoiRIfm6jjshUQ_375x500.jpg',
 64.2742,
 '7HWbnKehTfCIM8TuWrwnfA_375x500.jpg',
 82.63838,
 '7Mg2GNfBRYOQ9k93KXbUbA_375x500.jpg',
 53.821503,
 '7tENqhh0TCGvFJXPoOtVAg_pl_480x640.png',
 53.01962,
 '82zoKXtKRI2rckdXlp8qZQ_375x500.jpg',
 73.211525,
 '83IFR-DYTsya3ypz0EI2TA_375x500.jpg',
 62.28411,
 '8fweC76JTOCBA5KcXNhX-g_375x500.jpg',
 53.54608,
 '8N8QHKpfRVuORKsxuCby8Q_375x500.jpg',
 55.658592,
 '8QmxazRUSnKBjOgrk9JW_g_375x500.jpg',
 59.248913,
 '8uo91oOvRISlLFyujjhEDg_375x500.jpg',
 70.70168,
 '90ZnJfpXSVuNGGMRGgHBjw_pl_480x640

In [7]:
comp = {'5AiX6TLtQdipN0eiU8BL_g_375x500.jpg': 0.5780016, '5atjIMW-Qt-7fWIJzk9mDw_pl_480x640.png': 0.5664629, '5DnkaVG5TW2Y215rTWkKYA_375x500.jpg': 0.5292427, '5ecc9yy4TKeL9eh82MyJQw_375x500.jpg': 0.49135223, '5eLWb1NySUKzKMvIYYctMg_pl_480x640.png': 0.5235716, '5EQttTmHQ6GjcPMmFeBvfw_375x500.jpg': 0.44937968, '5FZUOH_XQkyIwjwlQ9rpjQ_375x500.jpg': 0.52549535, '5gD25ppLQDGqJiSZG8Gt5A_375x500.jpg': 0.4822116, '5gKGBBiFTHiyrZTHKKbpvg_375x500.jpg': 0.50079006, '5Gn8SjTZQ9KlS6vNQAYFJw_375x500.jpg': 0.4975149, '5gX4A5QbTQyg-980jbk3zw_pl_480x640.png': 0.49640274, '5GZR3AxPRgiLtdK382X2rg_375x500.jpg': 0.4933033, '5G_iTZkqRRiSx2qnxo7-GQ_375x500.jpg': 0.47784698, '5HnI3RFvSSu7Hm9iOdGjnA_375x500.jpg': 0.4513859, '5HqFVpycTDikvwB8qcSPxw_375x500.jpg': 0.49307838, '5hz90xH8TZ-crlgitwPtMg_pl_480x640.png': 0.5977618, '5Iflj6NISZa9SvNJyOmU4g_375x500.jpg': 0.50708085, '5IphOlAtRGGxc9goQ4P-uQ_375x500.jpg': 0.4779474, '5iqUzVDRR4aPEBV-AizQRQ_375x500.jpg': 0.5668023, '5ITt9UhGRkCOt_NWirgEvw_375x500.jpg': 0.49270114, '5IVe34gyS9GUg8wEAeglCg_375x500.jpg': 0.5404656, '5iVKfLZZTzigaR-Hx-VFlA_375x500.jpg': 0.49702358, '5Iz-jDR8Rtm_YBn0O2ixag_375x500.jpg': 0.5004067, '5jHuQcKKTV2h3xoXDvMtFA_375x500.jpg': 0.5165774, '5jP0Gw-aSK6N_4Eo5p-Z9g_375x500.jpg': 0.52258575, '5jUji87xQ9qTjVbHjmdzVQ_375x500.jpg': 0.46014223, '5jvQoGoOTwKeZIQmsf36sw_375x500.jpg': 0.48060268, '5JZGJUJeRE-pnRMV1KvKtQ_375x500.jpg': 0.44047213, '5KEdXLY9T9W227PTvEfFBw_375x500.jpg': 0.4794107, '5KgnlX6uQd6Trcmi91DEAA_375x500.jpg': 0.43155682, '5KihAzFgSN2-ejvXmyucSw_375x500.jpg': 0.49278796, '5KjqVwh1QUayMaLJmAY4qQ_375x500.jpg': 0.51583743, '5Km4tYQWQSaXYzqM1smvCg_375x500.jpg': 0.49053717, '5krPBcI5QiSORHf6pnSlcw_375x500.jpg': 0.5180141, '5kzrRkXaQF-Y8aD58bDXtw_375x500.jpg': 0.50566465, '5LUlFm9LSceffSMRI6346g_pl_480x640.png': 0.5881263, '5m7zPaDPQQ2Xwd_2-klGlw_375x500.jpg': 0.5269843, '5MCaNSysTEubAM_uW853Uw_pl_480x640.png': 0.5729323, '5MDOoqMGTkSyF3GDXK0JAA_375x500.jpg': 0.5130795, '5MIw-MV0TfWr3K-musxxPg_375x500.jpg': 0.55717725, '5mNMRuJ5R8qvoo7pYoL7rQ_375x500.jpg': 0.45383495, '5mPQe7nmRoOZ7dHJATV6pA_375x500.jpg': 0.5454393, '5Mv6Qv4zS6SCssef-R6J_A_375x500.jpg': 0.50468415, '5MwvgD43RqqISG2BYZz5Rg_375x500.jpg': 0.5248591, '5Na6sTsHSvSWF2NoE1SVlQ_375x500.jpg': 0.53151536, '5nMp85I3R7uxSF-IbAtaKw_pl_480x640.png': 0.52481776, '5NPNjvF6SqSXv-212j1kxg_375x500.jpg': 0.5000959, '5NvVj4qdQfiBI0-Pike8fA_pl_480x640.png': 0.5656559, '5nXcmGupR3yUmjKeidODeA_pl_480x640.png': 0.5393965, '5NXof0t4Qu6yUBzBASEbVg_pl_480x640.png': 0.5377276, '5o9QB9PDT7Kn_Eht5GSk1A_375x500.jpg': 0.5130132, '5oym2ZmqQlSjXJQ2uAqD5Q_375x500.jpg': 0.478769, '5pPkshuETB2YUEZms0bkBg_375x500.jpg': 0.47284475, '5Q0UuNiRRsmuhC-MFXSWTQ_375x500.jpg': 0.5397101, '5Q6RE6coSumOXGT3yW7aTg_375x500.jpg': 0.5268134, '5Q9fJaOER_mGEbXSSzTCKQ_375x500.jpg': 0.48668575, '5qBOZ5URTkC3ynFeAkqavQ_375x500.jpg': 0.5043842, '5QUq-i6fRE-qbN3G-ZnlzQ_375x500.jpg': 0.47458622, '5RcAmLXUQAOEe4z_oK93rg_pl_480x640.png': 0.48250517, '5rdET50aT8u6qSr0ZZdQYA_pl_480x640.png': 0.5474729, '5rUDiBn5RFaKE5ur0lKz6A_pl_480x640.png': 0.57536995, '5RxR1KG6TseMmtjul2AmMw_pl_480x640.png': 0.5186885, '5R_9qbA4RxCA0_Thj69dCg_375x500.jpg': 0.5220939, '5S6cYRr2TxaDoTbcAfrRTg_pl_480x640.png': 0.529789, '5sjpUSqRSi-mZKzfqdhPIg_375x500.jpg': 0.45494983, '5spsL2elQtm-KZX4TH4WEQ_375x500.jpg': 0.5128331, '5sy7d3PbRJy_USn0I_msXQ_pl_480x640.png': 0.43471837, '5tdYDMr2ReiQZRSk4iZwTQ_375x500.jpg': 0.5191759, '5TnM_8_ITGuaHoi6JjKbZQ_375x500.jpg': 0.4840574, '5Tofu0QISXeyzxeSlthQZA_375x500.jpg': 0.48457092, '5tUmjzxyQGuRYsBsAh_O2A_375x500.jpg': 0.55022126, '5TwBiSi2Sj-pwoqMMDQ7bg_375x500.jpg': 0.4782446, '5Tzf6fgCTpWOGHBw90Iewg_375x500.jpg': 0.5236266, '5UEzfBLfRk-7Hi2lKu8vtg_pl_480x640.png': 0.55034655, '5uJzWBQTTjW8Wx8XpzF05A_375x500.jpg': 0.43324998, '5ULTwTJWR32DTcg8ws9Hhg_375x500.jpg': 0.48603076, '5V3iFhOjSzWBm0iPEk6wug_375x500.jpg': 0.51202804, '5vi8-UCLRAG3Cgm-UM0lSw_375x500.jpg': 0.54084337, '5VmmdPFTSD2A21XoM3Zsag_pl_480x640.png': 0.5419824, '5WajtybcRka_lo5YWOr3mw_375x500.jpg': 0.45015404, '5WhxivgiTC2ZFbgUdp5vOg_375x500.jpg': 0.5122438, '5wIETnXURReitajsOB0Wrg_375x500.jpg': 0.5018461, '5X-AhphbS06wSRL5akwnGA_375x500.jpg': 0.5312105, '5X0X3oQhRsyxqaM2PdkS1A_375x500.jpg': 0.4313737, '5x3CbCiLTsKQkYKANdaC1A_375x500.jpg': 0.44431698, '5yd5ocI4SBGCWqlHGw1Qjg_pl_480x640.png': 0.5535753, '5yjO7PfySHOxO0yDqkbCBw_pl_480x640.png': 0.5387822, '5YqfOTxOTSSPj-Ccgw0iWw_pl_480x640.png': 0.5433869, '5yQN4JJqTWuh_soyTJj-Mw_pl_480x640.png': 0.52623075, '5YqOldTPT8agoOG0vlKhTw_375x500.jpg': 0.46320337, '5Z47y2etTKO-YEDFXmRmOw_375x500.jpg': 0.50311303, '5z84tKqoSsKHwJanm7V89Q_pl_480x640.png': 0.57997113, '5ZWlxCsjSca06FaAHAPZgg_375x500.jpg': 0.5008581, '5zxduNvCQc-jnd_5Us8D3g_375x500.jpg': 0.46237373, '5_YcywUfQX2eVs0-51kvBg_pl_480x640.png': 0.4842837, '6-EdI7okT0yrkBxiQBPWvQ_375x500.jpg': 0.42162883, '6-Hv8QMpS_qEmNcH5xXPDw_pl_480x640.png': 0.4849025, '6-QmgEQrQE6ssSLv8EwL3A_375x500.jpg': 0.45174333, '60_Zs4JeQOWh6bZs_TNBKg_pl_480x640.png': 0.5062857, '61N66eQORXijyRyvnvlURw_375x500.jpg': 0.49718183, '62-QNnBVRRubz3iDPxQLEA_375x500.jpg': 0.4896914, '62brRq_XQ5q5AQljUvf9eA_375x500.jpg': 0.5239542, '62YC4OAOSyG5aG-6faSRzw_375x500.jpg': 0.47512782, '63Gn1mQ3QjKotaQvcoIWjQ_pl_480x640.png': 0.55596876, '63kjTcIfRDuhvyrZZd8HNA_375x500.jpg': 0.44344178, '63TlPHGBQXiZASmlDMscJA_375x500.jpg': 0.48812222, '64-JovKrRO24Unzt0yFSgQ_pl_480x640.png': 0.5522481, '64gewmCgRR6K4X2JfET6_A_pl_480x640.png': 0.52797157, '650MOWoAS5epIXo5BjWrXw_pl_480x640.png': 0.59251046, '65mQnEKKSzeNv-PHgT1E4A_375x500.jpg': 0.5041713, '65S9-z_zQoOoY1rBg9J4zw_375x500.jpg': 0.53831, '65UyhKocTMKvMXnlXJpJSQ_375x500.jpg': 0.4973257, '66HMIFZTRoKMSGDhcboU6g_375x500.jpg': 0.57940716, '66xIquyPQE6xea3HmiAW-A_375x500.jpg': 0.4990101, '68nt9WfHRdasmBFzYB757w_pl_480x640.png': 0.5469826, '68YuPsUuT4ymqSd8tDYcAg_375x500.jpg': 0.51965773, '695UnZlXQqGQYX0v69R20w_375x500.jpg': 0.45738065, '69C-dQSeSJen_rceLcQaDQ_375x500.jpg': 0.5407143, '69rKqk4BSSePDbqKiShrVg_375x500.jpg': 0.48460153, '69wt5oXvTGmgn9LBAU9bFg_375x500.jpg': 0.5133656, '6AbDSujlR9KIKw6QHqqzbA_375x500.jpg': 0.49701616, '6aeEdCaLSOy67wl-D4uc8A_375x500.jpg': 0.50507724, '6am0f46pSumleUqUaMEFkg_375x500.jpg': 0.5112423, '6ASDtuE5R6C047o_J5VtyA_pl_480x640.png': 0.519416, '6B4x7yGPRLWfQBJjm5tmZg_375x500.jpg': 0.49306837, '6BbQB_I9R-2Kc5YhPp0TMg_375x500.jpg': 0.4367212, '6BI01n1RRAOL0n_T9rRh_A_pl_480x640.png': 0.57437164, '6CplBeRMQBWEHpQL0YlEZg_375x500.jpg': 0.5826853, '6D36ND9IRjSi5Oq3UIOMFg_375x500.jpg': 0.4621294, '6dZ44MdaQvSrGM-rgMvzMg_375x500.jpg': 0.44259924, '6ED1aBQAQaO_njJ8UdOb6A_375x500.jpg': 0.5181722, '6EfkaERHTOarj49rxvYDMA_pl_480x640.png': 0.596908, '6EnYxZCFQSaZPxyCgbBDoQ_pl_480x640.png': 0.4722525, '6EP0NhImS-qowmgUTY0isA_375x500.jpg': 0.47771892, '6ESukGUzTqq6coxz1koLPA_375x500.jpg': 0.49149615, '6EUeXhj-Sa22VgyxclW82w_pl_480x640.png': 0.56970453, '6f4BpidlQ4KXU72aFJsA1Q_375x500.jpg': 0.4369997, '6f6HaTcUR-GgRcICiegfww_375x500.jpg': 0.5147698, '6fjsi_HqQtyVdUGrv3-3_Q_pl_480x640.png': 0.53493273, '6fMh93JBS4uB50IfheK_sA_375x500.jpg': 0.5296588, '6fSnisG_Rq-KlTpUBlx-LQ_375x500.jpg': 0.5184381, '6FtMwE3DQ165NwmNrnwx3w_375x500.jpg': 0.5208048, '6gKMQmF1QgORfjj-q71Afg_375x500.jpg': 0.4944882, '6GPIoJ3uQ3GapDkg1JkE2w_375x500.jpg': 0.50958574, '6H-e9C7sR0y8J8mk_fO-jg_pl_480x640.png': 0.49177486, '6H6XCI90Tfmv5EgL44-MgA_375x500.jpg': 0.48032546, '6h7163oYRAyZOTYvMjGkzA_375x500.jpg': 0.5173312, '6HcedwZ8R7y29zcoC6Z6Vg_pl_480x640.png': 0.5205487, '6HPwCgACSGmFGNVwuo8mDQ_pl_480x640.png': 0.58045065, '6huUWiubTJibnytRUPCmxQ_375x500.jpg': 0.4520623, '6iW__1OkQPy7gF39V7XRcg_375x500.jpg': 0.5044776, '6jCWMIPLT9aX80SfwffnZw_375x500.jpg': 0.47959843, '6jqMPunZS0qc7DJIXYFk7A_pl_480x640.png': 0.51467013, '6jYx5nrsQ-K1HWHcpsl2cg_375x500.jpg': 0.48111683, '6kaItgLnRsOaAsgVuwJowQ_375x500.jpg': 0.4602484, '6kD7wAQ8QFivU3iUnge6sQ_pl_480x640.png': 0.500241, '6kiXNn-wTsGZU1PauRrluA_375x500.jpg': 0.50250065, '6KV9LFmjTeCtqXaF_jf0hQ_pl_480x640.png': 0.520843, '6KXA38uvSeC4xs4flJb9QQ_pl_480x640.png': 0.5536532, '6lcQgCsdSjK75vr9Psxfhg_375x500.jpg': 0.49303707, '6lcRlXOGQ1Wah_yuvHTsaA_pl_480x640.png': 0.5057558, '6lll9DK9S82Fv9PVuUDd1w_375x500.jpg': 0.53667057, '6m0cAXPbQvOWBKdXBzFAUg_375x500.jpg': 0.4768137, '6m881yaDQQm617rn8tRu4Q_375x500.jpg': 0.46063527, '6mk4BsCeR_q4_8scvmIF7w_375x500.jpg': 0.51776904, '6Mm14SSmSZ-M8nY_3R_zBw_375x500.jpg': 0.57541454, '6MRT8HU_Tw6FbbeqGP3JRw_375x500.jpg': 0.52713, '6mz4DQ2eRjaeZvk23P3E0g_375x500.jpg': 0.4303437, '6N8bmOxORRqCspuaq0nOhg_pl_480x640.png': 0.56085724, '6nDmGZd0Tk2AmZgWVWDEBQ_375x500.jpg': 0.48234668, '6Net9RWNRSibUVFbahiWlA_375x500.jpg': 0.50171095, '6nnt2QSgSv6kYzq4AfrzLw_375x500.jpg': 0.5075635, '6Nqq2cxdTnOkAuwqF9QqVg_375x500.jpg': 0.52933526, '6NSKVGhUTD23wVfODbPj1w_375x500.jpg': 0.47875375, '6NVn6PD9SVi8zQxsLKiZoQ_375x500.jpg': 0.44793212, '6OB5ZuzhTd-1IiIXmQjnLA_pl_480x640.png': 0.51664245, '6OEpfOzlRz-sP4cXSzvS9w_pl_480x640.png': 0.56905407, '6OKgHBCUSmqTAHFiXr4mnQ_375x500.jpg': 0.46317178, '6oKisclLRFKlMgX_5xMzwg_pl_480x640.png': 0.51820767, '6olW0uawQWO82ck1KJDA6Q_375x500.jpg': 0.48757872, '6OVDqNzrTSCL70QUyll8Vg_pl_480x640.png': 0.53981394, '6P0K2ew_R1OWoiLQ3m9PUw_pl_480x640.png': 0.4975361, '6P2t-a2DR868jR_zuX7qLw_375x500.jpg': 0.5088029, '6Pcrct2pR1We2OThIPdljw_375x500.jpg': 0.5539539, '6pIxNS7KQmSJwxfkmZ_TVA_375x500.jpg': 0.4996864, '6PZHuuziTGyXjIVQ08gBsQ_375x500.jpg': 0.5306595, '6q8o5aLYS72ra-DIjThgOw_375x500.jpg': 0.47185728, '6qaoN0YlSAm3N0nGIx_eTQ_pl_480x640.png': 0.54376644, '6qUDkn6ITLC2_4_JP8iPjA_375x500.jpg': 0.49202427, '6rDrCiOWQ4-cejgtUJOtPw_375x500.jpg': 0.45093292, '6RK-duFpRBigvi0z9neZSg_375x500.jpg': 0.48707882, '6Rsr1J1rRS6tnS_MUzhpvw_375x500.jpg': 0.56599426, '6sbrzO6xSWCulPS4NTgJ6w_375x500.jpg': 0.5145627, '6SJLQAhaQoOLHZf2VEcVbg_375x500.jpg': 0.5119593, '6SjqdSOeS0-OBsYbBeXNlQ_375x500.jpg': 0.50413525, '6sWNKDOCRpWp21fWeQkVSQ_pl_480x640.png': 0.47836828, '6tf6zIk7Q028JgSKf5lkjw_375x500.jpg': 0.5435658, '6TGqS9R0SxW1Lp5c4e9Lzg_pl_480x640.png': 0.49165112, '6Tnh0V7dQTauJ3xQaXAM2A_pl_480x640.png': 0.56698775, '6tYKYkW3SR6kmCsK4Egbpg_375x500.jpg': 0.4643724, '6tyN093ERlmHnj02kdjTeg_375x500.jpg': 0.5021155, '6U0mVoVPTZWJxl9Yp--fvw_375x500.jpg': 0.4455691, '6UnKTK6nTMe-x0h5_7lM2Q_pl_480x640.png': 0.51482207, '6v63mbmOTamf1iaUv88v0g_pl_480x640.png': 0.4896631, '6VD3X3iOT5u7KuSQiSUcGA_375x500.jpg': 0.40568188, '6vfAfTAnRE23J0_cZ54kkg_375x500.jpg': 0.4815838, '6vksB0TcTxCXpCZu2Cc-0w_375x500.jpg': 0.50731164, '6vRsXfpKT6q2dPJ8kLBuzA_375x500.jpg': 0.5331716, '6vuPe9nhQsOTdRFJHfzFcw_375x500.jpg': 0.5128791, '6vVF96o3QVyHxOCF5QmCGA_375x500.jpg': 0.48773104, '6v_S0MxYT9mltDMzrBqtAA_pl_480x640.png': 0.4998693, '6W6e0HrAR7-mWs5ehO90bQ_375x500.jpg': 0.593774, '6WqSaiqJQ0mPS0kvKZnLTA_375x500.jpg': 0.51112914, '6WSsf5xZQjiJzpAcY9Rvqw_pl_480x640.png': 0.51602006, '6WtXmMShRc2fFzFEUKL3PA_pl_480x640.png': 0.47337526, '6WuHP-K5RMKJZH0ROWjKhw_375x500.jpg': 0.4386496, '6wZeqBVNTo2neBjTJniiUw_375x500.jpg': 0.46014908, '6xjuh4jjQDKVdCCPTLe8UA_375x500.jpg': 0.48074096, '6xTDLa1EQQaWV82BQxBE8g_pl_480x640.png': 0.5025823, '6X_VF5EvQ_aX5fO8CPCaYg_375x500.jpg': 0.47172332, '6Y0bDKifSKm_-K_AP72Phw_375x500.jpg': 0.5808973, '6y3T-zVTS96w_CT07c9thw_375x500.jpg': 0.46123353, '6yAh-elFRzKcRNX788NFRg_375x500.jpg': 0.48907083, '6YK2_pDCTz-8yAse2zqf5w_375x500.jpg': 0.5029793, '6yVy_ITnT5CdjK0cZSYO2A_375x500.jpg': 0.50454944, '6yw8Z8OSShuwOBAEHaE9cQ_375x500.jpg': 0.4286653, '6yzHU_8_QoiL5hFY1mr8-g_375x500.jpg': 0.43780515, '6ZJZfLXbTB-13mO1tLHOAw_375x500.jpg': 0.47103807, '6ZPHOtI5Q9GPa1x21dmAbg_pl_480x640.png': 0.51252687, '6ZVGyNqUSfWYE-w3mMiGbA_pl_480x640.png': 0.49802336, '6Zz_aAUWSOCiktcPt3uDug_pl_480x640.png': 0.5135327, '6_ZanZFoQpiFHcjF8NwJQQ_375x500.jpg': 0.48412427, '7-4d20GJQb-Tz6v6PMjvOQ_375x500.jpg': 0.4733875, '7-ksc2KbRsaoRdXQIbbfbw_375x500.jpg': 0.49068666, '7-L5PhYFT1mOS4_TeYYfTA_pl_480x640.png': 0.50053144, '7-nZaL-BSL-O1_oQigMeig_pl_480x640.png': 0.51506114, '7-vIga9vRYSgO8RWPu-ANw_375x500.jpg': 0.47376513, '70uecbDxRmONYVSzbD4BzQ_375x500.jpg': 0.45874107, '71NiSknvQXizvTIaz7gsgA_375x500.jpg': 0.5184289, '7233hTCIRbi3TKoyQQaCtA_375x500.jpg': 0.44258347, '729xLveuSkS2bvupNgWmMw_pl_480x640.png': 0.51888937, '72hPgE9LQbCDJLZvbFJWlQ_pl_480x640.png': 0.5561069, '72NNZMlcTo6pgCrY9lWBEQ_375x500.jpg': 0.45372513, '72Y8e0pnScyOjqFY2_gCxA_375x500.jpg': 0.5125243, '732i-sVSTE2rlEPyr8tUbA_375x500.jpg': 0.5063312, '73Df9gg4SXW9-cw7DKkVeg_375x500.jpg': 0.48485005, '73dvyjnRRbSn1Jyc_cPa9A_pl_480x640.png': 0.5202425, '73m8VdPoQVORs24hz_JU7A_375x500.jpg': 0.49696428, '75g-R90vTcGoAXoaTrupdA_375x500.jpg': 0.53575635, '75nIZBKgQjKEALauAOK-Uw_375x500.jpg': 0.4731726, '761f0uF-TF-j2TtBpFyWKQ_375x500.jpg': 0.5281779, '76zLnYibTYuVEGhoulqyhA_pl_480x640.png': 0.50881106, '77anVe5LRG2aDl2LQskf-A_pl_480x640.png': 0.4887893, '77bB2kV6TqaZ1mrJ3OnJGw_375x500.jpg': 0.5260222, '77xRq5PvQ6GI1BIzLfFFKw_pl_480x640.png': 0.5268642, '77ZW45yGRQOx-p6YPvn7eA_375x500.jpg': 0.4585332, '78evV7tDRX2YdfZNGJd4NA_375x500.jpg': 0.50684804, '78leZgkXSc-G6GzO0MeB-Q_pl_480x640.png': 0.45560643, '7ac09wN1QuiM_SRjihunjg_375x500.jpg': 0.42643502, '7AiySLKbTF2cmoMug9zhuw_pl_480x640.png': 0.53674144, '7auBtzu1Tqi8ULQDFYQpVw_375x500.jpg': 0.47881866, '7a_LWl-bSoOC-IAlQ-kXWQ_375x500.jpg': 0.5262806, '7b6srBxeSYi-rrJAtgOjMg_375x500.jpg': 0.4990713, '7b8EkljjT82jTKJAMZUTRw_pl_480x640.png': 0.55428064, '7BhB-Pi9TA-uziwvkLqEhw_375x500.jpg': 0.49681172, '7bJkdXtrQrq2etrGlHia8g_pl_480x640.png': 0.52411693, '7BoYQ4w7SiaEFqNuAUukWA_375x500.jpg': 0.5019677, '7BTILrnMRVm5-GTQOW56jQ_375x500.jpg': 0.45469207, '7CALPRmXSOC9stHgrOCUuQ_pl_480x640.png': 0.52892846, '7CgYtJw4R6GDt1B2UBQKTw_375x500.jpg': 0.5267051, '7CVBwpMKTjOf_V8wVykgmQ_375x500.jpg': 0.4800349, '7D4XG_R9Rhi-Chg7OcMUNw_375x500.jpg': 0.5157443, '7DkICSnCQ9WAIode7iOs1w_pl_480x640.png': 0.53308237, '7DoXbJX2QE2eWveh6s0aOw_375x500.jpg': 0.49271613, '7dZ3Cg0VSoiRIfm6jjshUQ_375x500.jpg': 0.41325158, '7E6wnSV1RKCFQmtUApoIBg_375x500.jpg': 0.5022879, '7e77ifrTTjub1-jcY5vTvQ_375x500.jpg': 0.47010747, '7Ed8-uKbTjiZTLhZDQt9IQ_375x500.jpg': 0.52047354, '7eGE3fcnQYKUnobzDnKTpw_375x500.jpg': 0.46183905, '7ENLLuwrSzW_g3lAnfVBwg_375x500.jpg': 0.5040704, '7ephexNJSn-z-UkIjUwMOg_375x500.jpg': 0.4616666, '7eSEXLcQTrSbxPP89NYLXg_pl_480x640.png': 0.5283828, '7F0LWheDR9-kiJuC-oOBmA_375x500.jpg': 0.48421365, '7F3wuNO1QCibo19Bcc_cBw_375x500.jpg': 0.52480996, '7fB1DM7bTH-yRWuUdDK9Vw_375x500.jpg': 0.48373735, '7FfnrtKqSs253irxE7msig_375x500.jpg': 0.6260933, '7fGTZJZiQSOWQPp5K_GyNw_375x500.jpg': 0.53254133, '7FoiCeo9SEShbd-ZtiRaCA_375x500.jpg': 0.47532925, '7fQndAIMQcCHCfHTahNe0g_375x500.jpg': 0.45419803, '7g8Uuzo1RfyTnJOq8dFGHQ_375x500.jpg': 0.526283, '7GbCU0xsT5q-wN3jHEdS5g_pl_480x640.png': 0.51684636, '7GvwVQqhSE2oS3udtRpI1w_375x500.jpg': 0.50165516, '7hoQwVyaRpOS4V9oqYereQ_375x500.jpg': 0.4861975, '7HWbnKehTfCIM8TuWrwnfA_375x500.jpg': 0.47231817, '7i-cJfdnST63VwK2me7Ewg_375x500.jpg': 0.5254026, '7i3TbaVESeuqyGIfHLVw2w_375x500.jpg': 0.5254363, '7IFzylFMQ5SzIWwbDV27xw_pl_480x640.png': 0.50312227, '7ImKFBtRTXW3O1BKzP2YOg_pl_480x640.png': 0.61934143, '7imR5hh5T1WtO6hA0EeafQ_375x500.jpg': 0.5123114, '7JaQa32yTy6f9tuD7oAXxA_375x500.jpg': 0.4940022, '7jMv2DfRSLyTJrOc7s5NYw_375x500.jpg': 0.41361067, '7jnlnLbZTWqxnffLaKP7CQ_pl_480x640.png': 0.5609812, '7jSKOEvKQzakdx-Hdteh-g_375x500.jpg': 0.47698495, '7k5dyRSJTfWYIfBmDt4vag_375x500.jpg': 0.5647859, '7kQn3q7rQDqx50EC3UiLJw_375x500.jpg': 0.54091, '7KqYGIk2QBSL68k3aGi7hA_375x500.jpg': 0.5784213, '7kV4XyD0Tc-P6pDVEiYrPQ_pl_480x640.png': 0.51406914, '7KvXOhxaQ4mKzkVUXt2LAw_375x500.jpg': 0.49568322, '7lCa3h83SZSTEP6SsWFk0Q_375x500.jpg': 0.46400946, '7LhYDiqyRdecVYvT_RPaxw_375x500.jpg': 0.4675236, '7lH_wkKeQ-O7bVVHatEfhQ_375x500.jpg': 0.52219146, '7LQ7fh7hSQ2IzQNm6rPkBQ_375x500.jpg': 0.45138726, '7LTG9qRiRciupJOWgrK2Og_375x500.jpg': 0.49188614, '7mEHILJQRCmDiKVIejrRXA_375x500.jpg': 0.5297138, '7Mg2GNfBRYOQ9k93KXbUbA_375x500.jpg': 0.4848391, '7Mg9dDhcSHeDUeWJ3eZFVQ_375x500.jpg': 0.48610651, '7MGl4eKNQtqhGpEgo0g_5w_pl_480x640.png': 0.4963009, '7mLQ8E8mQDGM_8c4TGiK4Q_375x500.jpg': 0.557528, '7MW1P0luQ8iMoKxqj2HPkQ_375x500.jpg': 0.50845546, '7NIwIWplQoyv94M5-ebnDw_375x500.jpg': 0.5335368, '7NR5z5XCTGS_hgyblMn1xw_375x500.jpg': 0.49847615, '7nvMdFesTOypq-2g2lecCg_375x500.jpg': 0.5150898, '7NXr5TzYQvWev6Gm7JM9HA_375x500.jpg': 0.5096331, '7O-vXSRDSyCPl8Cs5aklZA_375x500.jpg': 0.52856684, '7ocRRCIHT42cgvQE-B0vVQ_375x500.jpg': 0.5213928, '7oNHmoHzRPern6T5HnJ7Mw_375x500.jpg': 0.4506945, '7phTbNQcRJ6YNQHuRuaqfg_pl_480x640.png': 0.53453344, '7PYdpT6lT1mfLaNztMdNFg_375x500.jpg': 0.44686934, '7Q7czt7dRDud0EM_eD60tQ_375x500.jpg': 0.43963832, '7qouFvhaSASbCRAw-_ZKLA_375x500.jpg': 0.48075843, '7qvUjz_8SC-CFevs4ZCV1g_375x500.jpg': 0.5321803, '7qXmdiKYTNKRmaQwB06XHg_375x500.jpg': 0.47018987, '7qYzksFSR6qfmovSIOy-cA_375x500.jpg': 0.46927434, '7r4VJM8JS02eqgYhmOuP6A_pl_480x640.png': 0.47311422, '7R5pmZ2eQWyelas2nlwc9A_375x500.jpg': 0.44568673, '7r9QKEZlQ5q-rwu60vXUcA_pl_480x640.png': 0.58235383, '7RcTNmW_QOSYcalbnsKBQQ_375x500.jpg': 0.5041363, '7RH5GdZRSu2L6ZFr-yp3rw_375x500.jpg': 0.4960717, '7RR5adtJTZmWM2DEY2be5A_pl_480x640.png': 0.5010796, '7rv1XtShSK6o5bRWKP_0VA_375x500.jpg': 0.52570236, '7R_W50zwRhe9gg0wG6cT6Q_375x500.jpg': 0.59297144, '7s2jUYy_SwyApcsTbraX4A_375x500.jpg': 0.56185484, '7sp-UZrmTPeS52aSw2ZW3w_375x500.jpg': 0.54857165, '7sQwNfDQQ1extd9ndxhtBA_375x500.jpg': 0.4960282, '7S_qHjQwSoypdmwc1qyCAw_pl_480x640.png': 0.49175692, '7tENqhh0TCGvFJXPoOtVAg_pl_480x640.png': 0.49371725, '7tKWiYIaSwqmerFsOU9uSQ_375x500.jpg': 0.47100392, '7Trdbi7ERZevz6LclliAuQ_375x500.jpg': 0.46447685, '7Tv2QpmBRrCW31MskF5Ejw_375x500.jpg': 0.52881676, '7U2fuC8jT3uE1Q7Y9qEiPg_375x500.jpg': 0.54881984, '7uiAwZGBTvakrWcpeFlf6A_pl_480x640.png': 0.54078895, '7uqGwwsjQcWypNzbG-wbXQ_pl_480x640.png': 0.5123999, '7uWWUCQxQjGuRxuf3MagVg_375x500.jpg': 0.48446104, '7v1EjcucRMaFoImbjrOA5g_375x500.jpg': 0.50885326, '7VchALgzQ-qETwrjOgw9Tw_375x500.jpg': 0.54594827, '7vgmFJpuRAOqzNR3AX0emg_pl_480x640.png': 0.49621063, '7vhO7BIHTBu9m_GKQdIKhQ_375x500.jpg': 0.539002, '7VMaL-3XSpieCkXWjJJEkw_375x500.jpg': 0.51378936, '7vQN48EUREy0y8H627XFXA_375x500.jpg': 0.48939502, '7WAu_pziTVieTycNYYQGIA_375x500.jpg': 0.50839424, '7wiVMsqkQJe7TUf_9RBgNQ_375x500.jpg': 0.4624879, '7WizE8e8Sdi9DAIuzS977w_375x500.jpg': 0.4820552, '7WnnqkMBQ6-qlMD7NgBLqQ_375x500.jpg': 0.5202708, '7woqo_85R4u76OZWNlx-hg_375x500.jpg': 0.47914493, '7wPd94raTu-N73O8RbtWZA_375x500.jpg': 0.48022074, '7wpUR4fhQXSWCkPo3pZEeA_375x500.jpg': 0.49896666, '7WsrYIg9T-uLME24W_k10A_pl_480x640.png': 0.563737, '7x0u_0vvS8SCZEWNDctEAQ_pl_480x640.png': 0.51907235, '7xD3wpktTIe0L7oHtC3BbQ_375x500.jpg': 0.49446338, '7XF8k6kmSyO6K4QefbtpAQ_pl_480x640.png': 0.5870215, '7yqH-4-MRBOgAru6Po6jbg_pl_480x640.png': 0.5353988, '7Zep7D83TTGhx-IZrIbRhg_pl_480x640.png': 0.56870645, '7ZsYxft-Rw-An74Nyk0eOw_375x500.jpg': 0.4533138, '7_KSGx1RQuyhuRChxzaUHg_375x500.jpg': 0.47588894, '8-I9WIeMQNi45tJayKrrcw_375x500.jpg': 0.49563503, '80qzNRGsRBOsLBBe01-f_Q_375x500.jpg': 0.52804387, '810R92BsTNWPTyQHlyncUQ_pl_480x640.png': 0.5606176, '8154zDLzSK-xRt5LAlD9ag_375x500.jpg': 0.48648682, '81lps_dESimTfWPrHiZ8Cg_375x500.jpg': 0.51138157, '81S1twMPTaqBxGIsacABkg_375x500.jpg': 0.54982, '821ZkdTOQ3acEwVS0CO5Pg_pl_480x640.png': 0.5334598, '82BTakJsRKKB1Pi90bpd7A_375x500.jpg': 0.47338495, '82i1D_daQeuXpo6zm1H1VA_pl_480x640.png': 0.48483452, '82kC_XiGTkeAgnTQ60-kkA_375x500.jpg': 0.48792362, '82qjBxEmTCecSkD3P-sANw_375x500.jpg': 0.5612133, '82zoKXtKRI2rckdXlp8qZQ_375x500.jpg': 0.4881408, '83Cr3KZVT9GTGjcKb4bitQ_375x500.jpg': 0.4379899, '83d3ua_JTEiN2AKkt-tG7A_375x500.jpg': 0.45791578, '83IFR-DYTsya3ypz0EI2TA_375x500.jpg': 0.54104716, '83ZvqTHFTKatcflV3kME7A_375x500.jpg': 0.53046626, '845HGhC-SdKnTy3y60o75w_375x500.jpg': 0.46422723, '852DkQSgSTyN2rnWgxIhDg_pl_480x640.png': 0.5242332, '85HXJM22Sb2q5voJEvRaGg_375x500.jpg': 0.5239458, '86mnDPhNRcCuuNo-DyGrmw_pl_480x640.png': 0.5031041, '875pzY5uT2O00xJTHs9HiA_375x500.jpg': 0.49774316, '87JZ4G8yTY-wJQv1mkudRg_pl_480x640.png': 0.50569165, '87kl2FaDTcyfC5ddKAPu_w_pl_480x640.png': 0.5115881, '88HlxQw_S_SGvO2kxyaB8w_375x500.jpg': 0.49728724, '88Nt2ElRQd6wQreRRAVA_g_375x500.jpg': 0.5369403, '88QhjAKAQJSObcBtqSKc1w_375x500.jpg': 0.45319882, '89fhtEPsTYmf-yBihY0QXg_pl_480x640.png': 0.57070684, '89hU1HHcR32ZEUsueb4ECw_375x500.jpg': 0.5125069, '89S7_B0fTRi0ppv9DlCo-g_375x500.jpg': 0.48601937, '89zQNUEQSwqfQrqp-5l7eQ_375x500.jpg': 0.47836494, '8aERyUM2TheuoesuemdUtA_375x500.jpg': 0.5326195, '8aoktZ-XTget6V6J8WDtoA_375x500.jpg': 0.4429393, '8aoL9rFVSAuH-K_saPOJkw_375x500.jpg': 0.50608593, '8aZSXKjwQp6n9uNHHbE9qA_pl_480x640.png': 0.51809543, '8aZt6rUXTSGLCksgawJJlQ_375x500.jpg': 0.4913345, '8b3W7UsMRDKWeVRBc0aHzQ_375x500.jpg': 0.5380959, '8B5-ArtbRQqlLRAEWs_vWQ_375x500.jpg': 0.48144293, '8b8iBRBnQVWN-lfw11iKtA_375x500.jpg': 0.492741, '8bP4x4BlQh6u5Gs0emtXZQ_375x500.jpg': 0.57312757, '8cbkRNcrQPORlK8PFTlyrg_375x500.jpg': 0.48896587, '8CPhMH_oQvqQSg5Tk8urRA_375x500.jpg': 0.47354248, '8CS62hQcTmu6jWc4rUrNqQ_pl_480x640.png': 0.52067894, '8cTNOXsQR4WIMNpNdcFjVg_375x500.jpg': 0.51124704, '8D2mjeThQ9ygeXq55vbDUA_375x500.jpg': 0.4447848, '8dUIKnMiTOOoirq-7Jn6Ng_pl_480x640.png': 0.5111569, '8e8h-sxgQBij1O2XpLMh-g_375x500.jpg': 0.53703266, '8EvmbwEpTKy6abNvcu7lqw_375x500.jpg': 0.49277493, '8E_m2dfXS3qN10aEodcZYQ_pl_480x640.png': 0.53991467, '8f31HM8CRZCs_Wnb0rDUJQ_pl_480x640.png': 0.56587833, '8F5s06BDS76w9Lleg0gZrQ_375x500.jpg': 0.51186556, '8F6T4k5NTae_aS5VgAncCQ_375x500.jpg': 0.54685736, '8FBApbLaTb-5sIKEMbaJag_375x500.jpg': 0.4551843, '8FeFQALmQ2eFQjvEQ8J9lw_375x500.jpg': 0.5074905, '8fTDsF_9SNSUa4ebcDA-tA_pl_480x640.png': 0.47228694, '8fweC76JTOCBA5KcXNhX-g_375x500.jpg': 0.53079695, '8g-8tI-oT9yKGAsHmspePQ_375x500.jpg': 0.4579905, '8H-Y4DlRQ4ufi1ORMUHW5A_375x500.jpg': 0.508544, '8HAKsFqfTHKvbsHxcWTI7Q_375x500.jpg': 0.47964928, '8hG5lsxUTPayPdKj0_Du3g_375x500.jpg': 0.50747, '8hWYctaASb6FzOSKCcaiSA_375x500.jpg': 0.55013454, '8I66tlqnSDO9jxd8G6R7fw_375x500.jpg': 0.5428519, '8I9gPrIaTuuKszstP3wa4Q_pl_480x640.png': 0.5700521, '8IaPL0U-RESIpjMZf3fBRQ_375x500.jpg': 0.5450644, '8ImZ0i9gTfCwa6L8FLDKOA_375x500.jpg': 0.4518176, '8jBra_zGR5aBCL5v4L_JEg_375x500.jpg': 0.52936715, '8jDTeUBzQVicFMKjnEM3-w_375x500.jpg': 0.48586005, '8Ke1sKOSRRuqzaxTA_e4KA_375x500.jpg': 0.50779253, '8KiBfz5NRhKj707SyF-I0w_375x500.jpg': 0.48258087, '8km-dyamSYWqRzQVgaCqbw_375x500.jpg': 0.4123033, '8kVzLG1jTbCEeHDnKxLuOw_375x500.jpg': 0.4575774, '8KZUDQ97StCuWRTszGkh4g_375x500.jpg': 0.5225922, '8lFwvvDZSVyL9aIBU3TV-A_375x500.jpg': 0.4692603, '8Lgx8_v1Q66uWXi0zwYi0A_375x500.jpg': 0.47033867, '8lSJmc4RR3en6mLWAipElQ_375x500.jpg': 0.50421685, '8lY4ZIMjTW-sfTEpizpJ9w_375x500.jpg': 0.5246775, '8LyCAeZgQBeqnlZus6-IBQ_375x500.jpg': 0.48713613, '8MfI1U6kRYSjs_TAXK89wg_375x500.jpg': 0.54925025, '8mGM_mqlRzmmYPdYNaLFOw_375x500.jpg': 0.46434882, '8MijhYUrS56jWEqKaNZfIQ_375x500.jpg': 0.5099021, '8mWIUOvJT96zOzGDLq15fw_375x500.jpg': 0.47813332, '8MZF-koaQ9mWhactJ89aLA_375x500.jpg': 0.45135352, '8n8e-ww4RAavJGK7wyyrZw_375x500.jpg': 0.46039864, '8N8QHKpfRVuORKsxuCby8Q_375x500.jpg': 0.47998312, '8NbG0X-bTr2JuKYMjNtG3g_375x500.jpg': 0.5722234, '8nU9yGIcSteKWpLwFfJGjg_375x500.jpg': 0.5136415, '8o5cXcZqQBmq9NHOlbiysA_375x500.jpg': 0.50971264, '8O7_alFLThOy6h7CgB4G4A_375x500.jpg': 0.52807397, '8oJGDZgyRWWAlX1vhDWKlA_pl_480x640.png': 0.50250834, '8OOM3c59QMSBqRiHa0PC5A_375x500.jpg': 0.57230926, '8OOtEyjCTPumR4oTk1C7aw_pl_480x640.png': 0.5313868, '8pBy8S4JSsOw4UdGWKkJFg_375x500.jpg': 0.51310027, '8PlFdk7VTDut26SxJj5Ixw_375x500.jpg': 0.5262958, '8PsyEoGtSkC6q7UvlAsJTw_375x500.jpg': 0.51031685, '8pUC4mrHQs-ihmtxSosIwg_pl_480x640.png': 0.5346163, '8q-pFWLFQhWEvnDTy6Rgxw_375x500.jpg': 0.47853398, '8QmxazRUSnKBjOgrk9JW_g_375x500.jpg': 0.47573137, '8QNAkDGhREqlXrZPz9GnDg_375x500.jpg': 0.48999438, '8qxfo7dWSvWM4qE-9vxerw_pl_480x640.png': 0.48261654, '8r2nHzlPSnyzqussDzL_UQ_375x500.jpg': 0.50830185, '8R71_-Y3Qrq6_enCx2efWw_375x500.jpg': 0.5085981, '8RHm7riMSPW7Fzre7C6AQw_375x500.jpg': 0.4766669, '8Rmk3QCxQVWnuQkZ8F-R4w_375x500.jpg': 0.45409012, '8RqWCMAtQBqYaVYgG80y7Q_pl_480x640.png': 0.53934324, '8scYXst9RTCSWecI2W4WaA_375x500.jpg': 0.49243686, '8SfTwVxSQF-qvz4lzurssQ_pl_480x640.png': 0.5425863, '8sG0fIR7QV2146ZAjTVsvQ_375x500.jpg': 0.48077393, '8sGzxmx8STyTcCD9IWBiyw_pl_480x640.png': 0.59861726, '8spja_HnRe2aKN98WIlCXw_375x500.jpg': 0.50375956, '8suGCsZ9SN2PC-0BDHcpZA_375x500.jpg': 0.46887487, '8T3pP_q5QVmFpr8GpkPebg_375x500.jpg': 0.47519884, '8TbCA1HxRqKVy-leHxFmWQ_375x500.jpg': 0.50378215, '8tnH7xsiR4-CsIA1iffnDA_375x500.jpg': 0.5119939, '8TPhfYGzSxOi0iJ3Qt0fmA_pl_480x640.png': 0.5426641, '8UbKKmYpS8KMEsy4LJj3YA_pl_480x640.png': 0.5628817, '8UD2eds7TIu9YJS56RGabQ_375x500.jpg': 0.4661623, '8UKTOMa1RY-F8VRjukh4LA_pl_480x640.png': 0.56639445, '8uo91oOvRISlLFyujjhEDg_375x500.jpg': 0.4738046, '8uQigGNeRwqELNKktx0ehQ_375x500.jpg': 0.5063171, '8UsmgjrjSGm7pEFciGByKQ_375x500.jpg': 0.4341446, '8V-Vtn-PRgiBJKzOBa5wow_375x500.jpg': 0.51652235, '8V8fic8tTuGJkGErjRUArQ_pl_480x640.png': 0.45327568, '8vbmj6gcQq2Y_C6DHtskfQ_375x500.jpg': 0.46409568, '8vet7lRnRyCKAYjbMNFVlg_375x500.jpg': 0.4983237, '8Vh6eNZLQTSxub20-Wb9Hw_375x500.jpg': 0.5557076, '8vM9SL8PRlaGckPb4SRnjQ_375x500.jpg': 0.48268634, '8VqN9kV-TPuDJ9QY_lZJUQ_375x500.jpg': 0.519259, '8VuSHcRBQjyaLG_5FkdAOw_375x500.jpg': 0.47354952, '8VXIkEYBQt6xdBmTmKEoZg_375x500.jpg': 0.49022326, '8w-ohTrKRFG7QMSBO8Gnhw_375x500.jpg': 0.53459126, '8w0JW9C1TQeRv6MUVj-SvA_375x500.jpg': 0.46910226, '8w9BCdVDRKGksbmP3EuCJQ_pl_480x640.png': 0.55680025, '8wdQQ1vpTLaqxvzxkeYuig_375x500.jpg': 0.47701818, '8WgYnfZ5TcauOD7nhVOdbQ_pl_480x640.png': 0.55265594, '8WLFikHxSruqt8E0_-37Og_375x500.jpg': 0.4934261, '8WRgtOD2Tqe4bnu_2hEU8w_375x500.jpg': 0.51772684, '8Wr_QvCJRDGyQqYIXgGX4g_375x500.jpg': 0.48680073, '8XN5DP3NR4GiV597hNscnw_375x500.jpg': 0.44135982, '8xQP68c4SeyJgPedzLMDGg_375x500.jpg': 0.54137874, '8YMTdlpgQqe44V4MVT9E_Q_375x500.jpg': 0.5061648, '8YPh732oRkqr9sxNJfuOsA_pl_480x640.png': 0.51760894, '8YsLqH_iRjmdS890QWl1-w_pl_480x640.png': 0.55305827, '8yxp9k6NSoyQDkvTgeCCvw_pl_480x640.png': 0.5963857, '8ZDkNpdtSXWF64m3shWr6g_pl_480x640.png': 0.5619732, '8ZLcax1QTdC2bX7y5h_xHQ_375x500.jpg': 0.4691755, '8Zr2Sc47TZCSDbU3fs4zTw_375x500.jpg': 0.5251537, '8zSkl3dpTGW-T2dT8XiqBg_375x500.jpg': 0.40049922, '8ZZT4K7AR7-7B0up47pxUQ_375x500.jpg': 0.47910938, '8zzymJRoTdyuMTshtHX6sQ_pl_480x640.png': 0.56186306, '8_dtMx3yTEC3YKPl3NwetQ_375x500.jpg': 0.5122626, '9-0HA7dNSPCgmWNe-w5uSg_375x500.jpg': 0.4522331, '9-rbulx-RlafSx-hYQLXvQ_375x500.jpg': 0.5194063, '90Jm-GXxQ_-BY8WYDs4caw_375x500.jpg': 0.523595, '90VpVhpKQ_KFNT7mVTgVqw_375x500.jpg': 0.49916348, '90ZnJfpXSVuNGGMRGgHBjw_pl_480x640.png': 0.5188495, '91b8su35T-6nIviAKEYTpw_375x500.jpg': 0.49999455, '91bXzOzjTmqTxNN2hp9qbA_pl_480x640.png': 0.528488, '91ddhiVMTD-VNQ1k5fftWA_pl_480x640.png': 0.54381853, '91ludcKNRr2QUsXiGG_3BA_375x500.jpg': 0.4893042, '92r_cGVcSpORvFoO-xcxJA_375x500.jpg': 0.47561616, '93FRuP8xQrqqxKJUTQUQLA_375x500.jpg': 0.5229416, '93FTOxIsTSywDxAyfz07Qg_375x500.jpg': 0.5018941, '94pmPgb4SUWIlwhwp8RbYg_375x500.jpg': 0.43694583, '94wmNJnQR123sODRMfca3A_375x500.jpg': 0.5134767, '95j5Kh7CTX6c_tnoip25ZA_pl_480x640.png': 0.52743024, '95kYNNb2TIaNPFUzvgDvpA_375x500.jpg': 0.49533126, '96AbdSV6QJuuTaCGHCPUYA_375x500.jpg': 0.5479428, '96cMlwlmQAGmL8xC0CbnvQ_375x500.jpg': 0.4886475, '96czMJb-RlGmwF6KpbE-bg_375x500.jpg': 0.47940305, '97Tj2YMqRbSYoogPdVPYgw_pl_480x640.png': 0.47703215, '988TRwHnS5ezCMl7Y7BSvQ_375x500.jpg': 0.5504022, '98pHdDzRRN-YqyFrwOOYWg_375x500.jpg': 0.51995575, '9968FEzCQiK_ohpYsgAtwg_pl_480x640.png': 0.6152123, '99ACO8tFTROHF2dWT2by3Q_375x500.jpg': 0.5371626, '99o_ex63TdKNQZB4L3H8hw_pl_480x640.png': 0.5825518, '9aEUZHjuTZaDbY3BuVw8FA_375x500.jpg': 0.4983416, '9al-CwQgRN-FCZCM_3qWig_375x500.jpg': 0.5206398, '9ApDXCJZQUukvWl2m7mnxA_375x500.jpg': 0.4874294, '9aQF5Oi5SDuSVvidy2Qhjw_375x500.jpg': 0.47796634, '9AUy-uH3SYmZmkoP1wxf8w_pl_480x640.png': 0.4538699, '9aVENZyyTSedN0chrU4S-A_pl_480x640.png': 0.5279067, '9b7XeWsVSnKnPnQJR2xm0A_375x500.jpg': 0.40842813, '9BY_YTaaSnqTmDX06M9ZJA_pl_480x640.png': 0.50156736, '9B_Z0b89RbSrD8hBaBr_1g_375x500.jpg': 0.48082572, '9cOC4f1tQZignYLiJpWtcA_375x500.jpg': 0.4566798, '9cP8DujETP-leCvSpYni4g_375x500.jpg': 0.48000696, '9cybraXuSJqOozDS0ExoXA_375x500.jpg': 0.5189315, '9D7Z714OR7qju72Hr4X-ow_pl_480x640.png': 0.551903, '9dgHCY21TNSbfZWzSDXP6w_375x500.jpg': 0.47442344, '9DNfh2YRQDyLFgAQm3VGSA_375x500.jpg': 0.48726505, '9DoGZx8aR12GDTWMGDJ5wA_375x500.jpg': 0.41581178, '9E09wVazRxqcTN-bVaPRsA_pl_480x640.png': 0.5111336, '9E0Yy5CwS_2n1EYx_JefJg_375x500.jpg': 0.5389294, '9E1hYJFhRHqOJaFIXxaP2Q_375x500.jpg': 0.5174913, '9EBpu27pR3Sc96ikBxm5yQ_375x500.jpg': 0.462078, '9EnEcP9_S9uHqYQ4LwWc9g_375x500.jpg': 0.5640941, '9enOlAYfQPe8yLNHTgJmYw_pl_480x640.png': 0.54875463, '9EP3wxpkTWqDi1fDjGZzaQ_375x500.jpg': 0.49888587, '9F3iX1fVRMu-SolH0Klyqw_375x500.jpg': 0.44717887, '9FkkxmW0QtexDSo1yMPlAw_375x500.jpg': 0.4769799, '9FoUmWVrRKyeoN7ctWfrlw_375x500.jpg': 0.5389539, '9fVeesrYSYWkA5aLy7R30A_375x500.jpg': 0.4570093, '9g-FThZsSDO5m78J7fAReg_pl_480x640.png': 0.4974155, '9g4ViEZkQ-y57M6vi4SErA_375x500.jpg': 0.47893387, '9g8_8t_0Q1ubo35YVmdUGA_pl_480x640.png': 0.5148918, '9G96h5zJT_KWHeKstP5LCg_pl_480x640.png': 0.5004847, '9gfbIuIOQTmWeupZ_PYpCg_pl_480x640.png': 0.56229013, '9GfURXJ8TGWgqDVwV-JOEA_375x500.jpg': 0.5486818, '9GGA3ohlQdGbKcgsAOvt-Q_375x500.jpg': 0.5318273, '9gLPeLgRQouylFFHz5w0Mg_pl_480x640.png': 0.52387077, '9GSTgaZMR1CaM29DKQC5Eg_375x500.jpg': 0.51073015, '9gtdOgPiQvGBXUMa9g2vsQ_375x500.jpg': 0.5173309, '9GWRFYAPS9GO3z3uifQXJA_375x500.jpg': 0.54828244, '9hDEfTRrQk6nW2DVRwD9tw_375x500.jpg': 0.49581718, '9HhyZJcJS8qQ9X0K56UfKw_375x500.jpg': 0.5385586, '9hQHGPX0QnuptN6Xf6o_8w_375x500.jpg': 0.5421178, '9JhbD3-CSVqhUT3YiO3yLw_375x500.jpg': 0.46506795, '9K0z_PzCRrmxvpzw557CFw_pl_480x640.png': 0.55073655, '9k1nutlBQ_SRy9Pv3pQxuQ_375x500.jpg': 0.5194116, '9K4jfGh0TLKos10AC1NE6w_375x500.jpg': 0.50961226, '9KLQAyjoQuKK5BP694H9gg_pl_480x640.png': 0.50773686, '9l4OGwM7TealUHvqd4Sk3A_375x500.jpg': 0.46885556, '9lBOF3HLRXOsemExhUFueA_375x500.jpg': 0.524063, '9LbzB114SCOXSNRYu1oYTw_375x500.jpg': 0.5017956, '9LcplXO_Q-ShsGIV2Xi-VQ_375x500.jpg': 0.45660946, '9LeTjEdARE-xIPKOIEhZsA_375x500.jpg': 0.5129725, '9lf-6Z6ORXudct1qmrp3zA_375x500.jpg': 0.53701866, '9LMRcMeqQWujSsbN-BrTnA_pl_480x640.png': 0.52784795, '9LxbHcnOTryvTt2bbfG1Cw_pl_480x640.png': 0.48727524, '9MlXH5IGTl6TIlOq8qByvQ_375x500.jpg': 0.4886984, '9MpzG1gYRoWMr4dgkoM6mw_375x500.jpg': 0.52485377, '9n943BotR2WBW_LHUgqsqw_375x500.jpg': 0.47671226, '9NBNh9ZqTkmWYEofPOpP5g_375x500.jpg': 0.49691036, '9nEHyl9HRAawPi37dP7Gjw_375x500.jpg': 0.4652766, '9niAy7JHSiWnQcOyT2Pl-w_375x500.jpg': 0.53061384, '9NJswUgeRJu6o86rMugxQQ_375x500.jpg': 0.4874776, '9nqksAO-Q5OwTv8qW0CzNQ_375x500.jpg': 0.4490164, '9o03AYmnR0-9dOtNHAI49Q_375x500.jpg': 0.4791804, '9O8pnoFrTMKxhi_NEh2eTg_pl_480x640.png': 0.47572386, '9oh9bY2RSYuhmK3_O5Tvtg_pl_480x640.png': 0.48453164, '9onJZxFgSwuEnnVlEEuVQg_375x500.jpg': 0.49688828, '9paye273TM63Ha3ZCTUAYg_375x500.jpg': 0.44903257, '9PDidV1ORiKU7VJdtJszRg_375x500.jpg': 0.45372507, '9pFcFZleTA6PY_vvFWB-gw_375x500.jpg': 0.5017519, '9PUhrLWwQyerTUD5O3KjyQ_375x500.jpg': 0.53649247, '9px0ysGYSBuRLhcEzfSy0Q_375x500.jpg': 0.49989167, '9Q-oRI33QfWsP64mjsxumw_pl_480x640.png': 0.50057036, '9q374fUUQmqv8D68ZXn3Og_375x500.jpg': 0.50163937, '9Q577UTYQDmYCPlO12149g_375x500.jpg': 0.47775924, '9QJHyl8_QwGvtk2ZJtXiXA_pl_480x640.png': 0.50778186, '9qQkQzTtS1e3jswfkWvnxg_375x500.jpg': 0.50158465, '9QuuqRGlR7uK7kNYsLNsug_pl_480x640.png': 0.5420015, '9qvn6yDTTIeVMEEnWUomNQ_375x500.jpg': 0.44302282, '9rEN7sf7QtKFi4Q-e2lOnQ_375x500.jpg': 0.46398282, '9Rosq8sJQ4OOPGhVVQBUgA_pl_480x640.png': 0.56330776, '9RQeX55hSFGW4cTTsVv8yQ_pl_480x640.png': 0.51262146, '9RSGFFwzTtaMB4zJxxudUQ_375x500.jpg': 0.54156935, '9rt0halfTkSPtwl8UQ2NJA_375x500.jpg': 0.46644187, '9ruwFtkHSNGYepQmgd9dMw_375x500.jpg': 0.4986769, '9RvYksCOQr-Mi_v5XWby5A_pl_480x640.png': 0.51837426, '9S6kcFogRS6-WdAWiCbnpw_375x500.jpg': 0.4930045, '9SEIh_zKSm-n0cfhhdn5pw_pl_480x640.png': 0.5514055, '9SGNb14QTXOL-vM6p9XDBw_375x500.jpg': 0.5481649, '9sJbMlJmSCiFq2oZc4lPEA_pl_480x640.png': 0.5609098, '9sPBdpEmT-aCaX505aiaBA_375x500.jpg': 0.46811175, '9srJ1IwMR1K91tg-pUYWZg_pl_480x640.png': 0.54037565, '9s_rWACGTbmjWC2YtXksnA_375x500.jpg': 0.5357066, '9tFER-6wQdyzOszzIs91eQ_375x500.jpg': 0.54624647, '9u3I91h8RKuOlC5zR8XXzw_375x500.jpg': 0.5420842, '9U70spkoToGKLZR9IrVmvA_pl_480x640.png': 0.5551926, '9UGvCALoQPS6IL5T6tZzEQ_375x500.jpg': 0.4823606, '9ujlHaAlR4StDDTpS2J-wA_pl_480x640.png': 0.5927661, '9Um9RA81QjKjT1kmC5gtgg_pl_480x640.png': 0.51933527, '9UqnG81-RKy3Epu-e8k30A_375x500.jpg': 0.50030845, '9uuUloF8TCWZKkKF4OCGKQ_pl_480x640.png': 0.47044438, '9v4eT7MUSG6B6fcc_s8JNA_375x500.jpg': 0.53145814, '9v5LFsHoRvSn6o4TjEpeUA_375x500.jpg': 0.50204504, '9vrxmsnRQvKwXO433EOHYQ_375x500.jpg': 0.50701517, '9w9KnLJKQHusCNJCwZQ72g_375x500.jpg': 0.48081192, '9wfhhQxKQv6sldoJ8Jg4Ow_375x500.jpg': 0.5016478, '9wmsRtUXRs-FcUCF8aj9lA_pl_480x640.png': 0.537237, '9wThMLxDQvahJqBGWdpd8A_375x500.jpg': 0.52759653, '9WtLPnizTlCiZTwMugzKsg_375x500.jpg': 0.44780374, '9X8LwRw0T1Oe5EJ3hf5jmQ_pl_480x640.png': 0.5151508, '9Xc2MmAiTCmPj8-WV9MV9g_375x500.jpg': 0.50279266, '9XnEYBlEQLCIpMt031ArJg_375x500.jpg': 0.5228464, '9XptWCrORLqAhq7DeBMFKQ_375x500.jpg': 0.52701116, '9Xs-9UX3TriX1jopSo95qg_375x500.jpg': 0.5045927, '9XYHrFGdTZCPWtQNx3BX2A_375x500.jpg': 0.4473168, '9ykYYAaOQtmq9Gan10WY5Q_pl_480x640.png': 0.5816192, '9YN6p2SxSNG36OI-JVKHLQ_375x500.jpg': 0.5209586, '9YNeqopOSyehAzbiSvbjqg_pl_480x640.png': 0.50516033, '9yXLjq0wSOC7Cc9tInQ_Eg_375x500.jpg': 0.4406545, '9z1PzGbIRqaA05f0s0NLXw_375x500.jpg': 0.5218543, '9zPg7_ugRgqSK41OFCfL4g_375x500.jpg': 0.52652097, 'A-xQ-1QZRQCM79-3E8898Q_375x500.jpg': 0.53112626, 'A0iijDiQSQqxYP3uXByhTQ_pl_480x640.png': 0.53556025, 'a0ZgzEM9SJm_d-7x2p9reg_375x500.jpg': 0.525401, 'a15wfydBQTmqyvs1CAW7ig_375x500.jpg': 0.4924316, 'a2tDfAsSQxGNPauwo8eApg_375x500.jpg': 0.5279137, 'A2y-u2bVRE69JWRNoPhuUw_375x500.jpg': 0.45094344, 'a3eMPqfwQ2aUilh9v_M0fQ_375x500.jpg': 0.5110174, 'A3Vwz6fUQyuG6ZtM-gauRQ_375x500.jpg': 0.4882754, 'A4fJVOp2TP2AcBzNRAyzdw_pl_480x640.png': 0.5422359, 'a4svaCEzQTOB602cHKIi7A_375x500.jpg': 0.47744116, 'a4YCKfG5RgKZLpqoLy3CMg_375x500.jpg': 0.4728026, 'A4_eEiHGSjmKdgpnp-5xhg_375x500.jpg': 0.549491, 'a5FVr9O9T0WCJulM6SvnWA_375x500.jpg': 0.4771366, 'A5oOH2C9QcegAPeDa3pW9w_375x500.jpg': 0.4799791, 'a5OPREEQSCm85UH89Apepw_375x500.jpg': 0.43952742, 'A66tW2LKQPGIgoNyv5HzKg_375x500.jpg': 0.4906173, 'A7-6k4cYRzK4FZXRmx0qFQ_pl_480x640.png': 0.5417226, 'A7UUT_-WS6-PrNK1GQAELQ_375x500.jpg': 0.52389544, 'a8KnBarcQz2jTEtWhvZDFw_pl_480x640.png': 0.5479189, 'a8ofU1c6ROObArUz_zUOvQ_375x500.jpg': 0.53573364, 'A8SHIeoFTCCiEcH53U9uzw_375x500.jpg': 0.5032325, 'a9021NtHRQmwNspyrPhC0Q_375x500.jpg': 0.4632391, 'a9l4PwUiSKembmwa5_vxWg_375x500.jpg': 0.5329519, 'A9wXtVLdR-y-BkhKkOjtUQ_pl_480x640.png': 0.62825763, 'aaCxN0qeRsqYQTH20sJYOg_375x500.jpg': 0.51565564, 'aAigj64kTL2oUSzRhjkUBw_375x500.jpg': 0.515008, 'aamOCxM0RhmWwX-GDAA5-Q_375x500.jpg': 0.5353256, 'aB4RMSdZSouD5ShmA4T6zA_375x500.jpg': 0.50192237, 'AbCglbRFTVW7cV2djvY0sg_375x500.jpg': 0.50373054, 'Abd_Xg8QQQKXcMcJbcsP8w_pl_480x640.png': 0.52964354, 'ac8MKwKURayCVUrKFM5VhQ_375x500.jpg': 0.5231241, 'ad-CgVL9TtyjIoNJNTMAVQ_375x500.jpg': 0.5125096, 'aDdgUPx6Tm2Z1j2eiluqOA_pl_480x640.png': 0.53956115, 'Adl_6OLHSXKqhEA89qRe_g_375x500.jpg': 0.4601168, 'aDPS-MfaQaCCko57aPsogA_375x500.jpg': 0.4780999, 'aDwdQdtpQaGl7pkWScMl_w_375x500.jpg': 0.56142527, 'AE6tuv2QQdSlTIwS7L_Pqg_375x500.jpg': 0.47491413, 'aECbWdVcTW2BmjU-lOeI9Q_375x500.jpg': 0.49390066, 'Aed_9i_CQ4uEYX9lvHOPEQ_pl_480x640.png': 0.48926017, 'AehBww3hTBid5cQViEnsrg_375x500.jpg': 0.46706873, 'AEhxJ-elSGGJTgQRBjXisA_375x500.jpg': 0.51489604, 'aErSJm5NQGu6OmkGPlTbSw_375x500.jpg': 0.49370444, 'aetKDlmIQRufgcn3VUiuaA_375x500.jpg': 0.5086097, 'Af1iJH_SQxu9P2eEN6KSrg_pl_480x640.png': 0.53050417, 'AfEQZmMESeW7P4i1hJh2kA_375x500.jpg': 0.44158697, 'AFhzplK2RJ6qqKgMVNZkxw_375x500.jpg': 0.4952789, 'aFozXqnlRWGVhP1LlKcTRA_375x500.jpg': 0.44973275, 'agAw0IHKQxGIIa5_WvZxcQ_pl_480x640.png': 0.49619249, 'AglZeSzHQ2aINE-kmtpgWw_375x500.jpg': 0.5549378, 'agNH2U1LRH2ncRDuGaQfkg_pl_480x640.png': 0.54853487, 'Ah4MCZ0BQ9-1Jak6hUWXkQ_375x500.jpg': 0.5417763, 'aHf9pGxLRh25VMvYfr8OOQ_375x500.jpg': 0.54332954, 'ahfa431oQHKd-ga1YZneGQ_375x500.jpg': 0.503937, 'ahopUoQBTgWC2ogOKvxTfg_375x500.jpg': 0.5232434, 'ahvcfbnRQzu1E2bvIvIZYw_375x500.jpg': 0.513469, 'ahyqUQa5TNWwr6vsU4NUSA_375x500.jpg': 0.5815496, 'aI37T2YrSyyseuSJpbECAA_375x500.jpg': 0.45801967, 'AIlpNM2fSWC7MnpPSSpz-g_375x500.jpg': 0.44914678, 'aipk5SJwQo69qtuGFCvQ7g_pl_480x640.png': 0.58009267, 'aisvM87DRsqhJ_q25eAmpQ_375x500.jpg': 0.5026084, 'aIVz5TDzQGiZex2vk7TjIw_375x500.jpg': 0.51933223, 'AiWMyScmS2uI76FKIoqBfg_375x500.jpg': 0.47589898, 'AIYfDyHaSIqEomv2gnyFZA_375x500.jpg': 0.5186728, 'aJ5BhlGdSqa-Huem32HRLw_375x500.jpg': 0.45281747, 'AJ5MoUfbTSCz060juCc3Nw_375x500.jpg': 0.5173435, 'aJXNrqgoQEi6MiTjJ_Z-ow_pl_480x640.png': 0.48551983, 'Ako1AePaTHiP1Ff_pLLYWw_pl_480x640.png': 0.49514973, 'AkRscq1OQfmqDqC6iCpk6w_375x500.jpg': 0.48012757, 'aKw7uzphQH2NhOsYOv9sjQ_375x500.jpg': 0.47965816, 'AkyQSDJWTSewQ9wJHf3N-Q_pl_480x640.png': 0.5046266, 'AlAn4nvYT3GWBtT7kTlfyQ_375x500.jpg': 0.41080362, 'AlytfdydQD27i6DNNpylxg_pl_480x640.png': 0.6303919, 'aMEVNT_uQTy_bBR3XejafQ_375x500.jpg': 0.47163835, 'amnhZxRIRA6iQ8-1clzGgA_pl_480x640.png': 0.56407833, 'aNBzVsVlQr-w5mYTljfZiQ_375x500.jpg': 0.47280836, 'anMyHIuQTgWkgEaN-CGLpA_pl_480x640.png': 0.47830376, 'ANQPpL8SR3qMzH1cWTG8yA_pl_480x640.png': 0.5401343, 'aOmXlrohQp6zcttCr7sZ1Q_375x500.jpg': 0.44056556, 'AopxmFy_Sj24klvk4U_dHA_375x500.jpg': 0.47499123, 'Apgvt85cSyu0k0uu5K96LA_375x500.jpg': 0.39990392, 'apu5lzyqQnKHjsJlaT933w_375x500.jpg': 0.5271959, 'Aqlrg80oQsGUHFJ4fWyzJw_375x500.jpg': 0.47606862, 'Aqon3YAFQf-rEU4Q263FPw_pl_480x640.png': 0.53762025, 'aR2TcJTVQfOnbKfpkFcKQg_pl_480x640.png': 0.55470043, 'aR33QHU8QBKHOhsEjDYQCw_pl_480x640.png': 0.5438392, 'ArFzQvxrQQa16lOQ3nGFTQ_pl_480x640.png': 0.57724404, 'ARKOBHopTRmE-c35qjoFXw_375x500.jpg': 0.46105704, 'ARmFx-MGQCS0vpfCnW_dpw_375x500.jpg': 0.38280064, 'ArRsTLwiRnSrov7WLFd0CA_pl_480x640.png': 0.51101583, 'Asp-WwLLTqWQyoSHLtZKow_pl_480x640.png': 0.54095405, 'asQQod-ZS12CsB_IXQodOw_375x500.jpg': 0.50992614, 'aSwmApsXQrO-ncEE40Cw1w_pl_480x640.png': 0.4942106, 'At1sDPg3ToOrd7nCRMeLWQ_375x500.jpg': 0.4638004, 'ATAdr2GBT16Sg4Z9rVVjhA_375x500.jpg': 0.5757189, 'aTCY2XFaSfyv4K4nMnT3Kw_375x500.jpg': 0.50723976, 'Ato6uM82RTa6-O8UIn1vqQ_pl_480x640.png': 0.5591871, 'aToltuM7SPibMa4LtW1kgQ_pl_480x640.png': 0.5225328, 'AtVe2p-fR5SKvu4Agp_TAQ_375x500.jpg': 0.50141746, 'AuHe7yxGTbS6lgNuAQ_MJg_375x500.jpg': 0.531158, 'aUMpsvlnQkCo8VpIxa1rkw_375x500.jpg': 0.49227726, 'auxRCdc_RK2xouxYWWPT5A_375x500.jpg': 0.4715107, 'AUXUbUoGQn-XDxCHjMqQeQ_375x500.jpg': 0.49059176, 'Av8Qzh5HRK2OA2kq2eV-Xg_375x500.jpg': 0.45231554, 'avgZWDiFQ7OiAT8tb5lRcA_375x500.jpg': 0.524123, 'aVqIXrhqTYSixEwNJbeABw_375x500.jpg': 0.49722943, 'Aw1z9LDGQKCduc9mck99VQ_pl_480x640.png': 0.5502494, 'aw20i_uITOSb916A5Fiavw_pl_480x640.png': 0.5715911, 'AW9Ti-3LRD2c6A7pcrjV8A_375x500.jpg': 0.50543976, 'aWCgL9W0RMm6m-oL4ELkcQ_375x500.jpg': 0.49261236, 'AWR7UZPnTLmp31rh5bxnZQ_375x500.jpg': 0.5042785, 'aX6prv08RayNBsOpuEzzgw_pl_480x640.png': 0.5544415, 'aXAfmV11TDilsgo6YrJQRg_375x500.jpg': 0.5571788, 'axfxwrCyS-ugLk_KBSzGBA_375x500.jpg': 0.4837312, 'AxMVXrwrRsiCcXCM8VFlbw_375x500.jpg': 0.5150836, 'aXm_0S2RSN6rYkswdCuTkQ_pl_480x640.png': 0.52378356, 'AXnEJS3XSRy34gIYBtoeKg_pl_480x640.png': 0.51635844, 'AXwgzGFmSwug5P0V2a9bRA_375x500.jpg': 0.5187355, 'ay-LHoeaTEi45PggSBAb-Q_375x500.jpg': 0.5226502, 'aY4hSLWwSHmITereK3GifA_375x500.jpg': 0.46961305, 'ayGAfsDGSt-_EGogrg4f0Q_375x500.jpg': 0.54982746, 'ayPTkkdnTtq5h0hjTykHPQ_pl_480x640.png': 0.495193, 'AZJsKGPtTqCS-s0mBStTBA_375x500.jpg': 0.41582924, 'AZJZYLaGTTmwUtl_7ewdfQ_375x500.jpg': 0.46017584, 'AZMlLdYMTGWvRLHVZOtlcQ_pl_480x640.png': 0.5014169, 'AzQ-B38PSbGvwUOq-JX4VA_pl_480x640.png': 0.5328099, 'aZ_wPxOJQWyPIMhOS_pRGA_pl_480x640.png': 0.5529716, 'A_4daarBQiipfYEoDNuO-Q_pl_480x640.png': 0.5295308, 'A_6xSvv7RmqM6I-ykvIpvQ_pl_480x640.png': 0.51674724, 'A_9k3yAjQ8-aDMP73-F-Gg_375x500.jpg': 0.5253615, 'A_KdxLe0SISMVn2oGays4w_375x500.jpg': 0.5401546, 'A_N7Xv_SQZqxwfKOE2ODvw_375x500.jpg': 0.5072928, 'b04NA0CrTlepWZQibVjRJg_375x500.jpg': 0.510072, 'b0BJ42IHRdOpQCeXDhuXwA_375x500.jpg': 0.5380283, 'b0E9SDUGQj-lAUQNQ4_3tA_375x500.jpg': 0.5586669, 'b166oJseSBizpmrnuIqYQw_375x500.jpg': 0.4966547, 'B1y_PfX2Sm67T_L6S917KQ_pl_480x640.png': 0.56201303, 'b2uLthH9TCmn5gELPCKN3A_375x500.jpg': 0.5207499, 'B2wo0XYYRj--ApiuK9UC0g_375x500.jpg': 0.4892933, 'b2YcdtOKTbWgmG0Drdx-Cw_375x500.jpg': 0.48722774, 'b30F5QKlTwuYcmvqUcnH8A_375x500.jpg': 0.4514758, 'B39_E38BSR6L81IGnStFhQ_375x500.jpg': 0.4640072, 'b3aGdma-RKCkZrsSNw_0cQ_375x500.jpg': 0.5068214, 'b3aIA393RNSFXuS50-wSVg_pl_480x640.png': 0.48229894, 'B3J2578NS3arfn5lHE6kyw_pl_480x640.png': 0.50053966, 'B3OpPCTgSDK_jktWAPK-Xw_375x500.jpg': 0.53308344, 'B40hacvKQduagWV-uKeDuw_375x500.jpg': 0.5178233, 'B44DyV6cSJuUr8ISETGgaw_375x500.jpg': 0.49766225, 'B4wOsQRZQhKbCS696moW5g_375x500.jpg': 0.5556875, 'B5dGcHsySnmkY4Wft-9sdA_375x500.jpg': 0.5118253, 'b5vTKIKyTGaIdpjEGuEzOw_375x500.jpg': 0.48254937, 'b6gzwXZeROa5ZPS45rKfWg_375x500.jpg': 0.47772607, 'B6i9T1mqSx6JDEFlQGmcEw_375x500.jpg': 0.49812618, 'b7eOG4azRRer-gNl_l75uw_375x500.jpg': 0.5084171, 'B7OFEftVQjyIklZDmgoF4Q_375x500.jpg': 0.4871564, 'b817Wf67Rl6auLjNmxiQNw_375x500.jpg': 0.47432002, 'B9LqfTGAQxGfk0Jn4gF4tw_375x500.jpg': 0.49474925, 'B9wIZE55SrStpSOVyVG8Nw_375x500.jpg': 0.4569874, 'b9ZK2FTdTN6_xzV-t_6uCQ_pl_480x640.png': 0.45836166, 'BA7LR-J6RqaMof1lpXSn8Q_375x500.jpg': 0.4531139, 'bAFoxaESRfyYD6xI2o4OYQ_pl_480x640.png': 0.5542984, 'BaoNlMunTlC54FA04rp-RQ_pl_480x640.png': 0.5027769, 'baooImnkSKuATy5ukeiUZQ_375x500.jpg': 0.50954205, 'bAxJUgTNRcSfbQ1WF76X7g_pl_480x640.png': 0.5767165, 'bBK_H38UTwOLBEah5XknCQ_375x500.jpg': 0.48895204, 'BbMHNEw1SRGKmmndaSj41Q_375x500.jpg': 0.5075993, 'Bc5J3E5bSAOlvD76yzQQUw_pl_480x640.png': 0.51965004, 'bCGRLA-8RViLsA9U-1fUSw_375x500.jpg': 0.44326362, 'bCpvSWHsRhSmEYXbaPKLXA_375x500.jpg': 0.47566164, 'BD52fHO5RJ6Y-qA9KvvAMw_375x500.jpg': 0.43792966, 'bDaYt--aQKmTtCuZkpn3MA_375x500.jpg': 0.45497137, 'bdEQMOfXTAGPVzojZnAPHg_375x500.jpg': 0.5522939, 'bDLn0vftSg-viSYqhIJ3Vg_375x500.jpg': 0.50814193, 'bdPo_yZEQfeB1tGf442YOA_375x500.jpg': 0.46631148, 'BdQtPayLThqb8ObhWIjBNg_375x500.jpg': 0.48542127, 'BE-XBf4oQIOKDrs2tJLCfA_pl_480x640.png': 0.5373605, 'bEien7xTRw6w5J-aOxEOxw_375x500.jpg': 0.5032594, 'BenaKQCLTO2aQcv7-xYUyw_375x500.jpg': 0.5013569, 'BfabvphHRBKY8-z9BcGKJg_375x500.jpg': 0.5129495, 'BfKj_7MWQSKZbWttAF3VfA_375x500.jpg': 0.50353956, 'BFmbypwySeW9hujaZ96tsw_pl_480x640.png': 0.5676842, 'BFsd_M5NR3S7JkDl0U19TA_375x500.jpg': 0.5186956, 'bG-SSqkLR-eOFliLzNR3Mw_375x500.jpg': 0.5100671, 'BG01Po3QRdiYSJ7FvuosOQ_375x500.jpg': 0.54158413, 'bg1i7vkAQn2_Oz-Vc87gaQ_pl_480x640.png': 0.53201765, 'bGbclYEUTMuo6keqlPmrLw_375x500.jpg': 0.48695204, 'bGIpT3ZCSLmewF2vjwzYTg_375x500.jpg': 0.4402921, 'BgNm0gfMRtmQ4yU5ryUMJw_375x500.jpg': 0.5047177, 'Bh0S5QInTui5U8eJakZXSw_pl_480x640.png': 0.48805007, 'bHf4pgcXTb-bpo6J_a0NUQ_pl_480x640.png': 0.4628855, 'BHgUSOYnT9GSOi-nLNkGPA_375x500.jpg': 0.49132892, 'bhniLfkrQ2u2xn58rhMa1A_375x500.jpg': 0.45703873, 'Bhy4XAJnTmiu5J1UN7ruCA_375x500.jpg': 0.47008616, 'bI4ObnT4TIehDopl6tlGWw_pl_480x640.png': 0.6410688, 'bI4Pfjs6RTeQqV8mrp2Vfg_375x500.jpg': 0.49357155, 'BIHxaArKQzyqhBL8xB2KRA_375x500.jpg': 0.56115824, 'BIplUIORSOW8JK9SnLoGXQ_375x500.jpg': 0.52344584, 'BjbCmrAMQoKL68JXf4XNHg_pl_480x640.png': 0.53030133, 'BjDrZ6f0QTGIbTSoIq0LAQ_pl_480x640.png': 0.54352874, 'bJMDtgEiSiqf7FnrY9DiqA_375x500.jpg': 0.5433445, 'bJprepk3QuqUT66gy-pZmg_375x500.jpg': 0.54398406, 'bJQQFaSRQ0GnatslEoyLcA_375x500.jpg': 0.5039866, 'BKegRS5nRty-1jJEanT1Dg_375x500.jpg': 0.47190627, 'bkJRgHVtQry55owOQCCvJw_375x500.jpg': 0.4406031, 'Bl9Uc-ZnST-OWCUWnhp2zw_375x500.jpg': 0.5069991, 'bLpi-G-nTPWxed2cZg6H3g_375x500.jpg': 0.53249705, 'BlPT9OvSQsG7t1YrU4SvPg_375x500.jpg': 0.5145188, 'blTbpewfSK2cUX1dH4k1Tw_pl_480x640.png': 0.5294908, 'blvF6C1FQlaH9Qv7zDhCZw_375x500.jpg': 0.48106954, 'bM2nxT2dRqKsGZpPQYE-mg_375x500.jpg': 0.4725097, 'bmCw1pm_RYOKt9ipk_A6Ig_pl_480x640.png': 0.6005985, 'BMqNt2FTRH6YARwUwZlcsg_375x500.jpg': 0.54492795, 'BNd7j9mpTKqUGsdcz1RvOQ_375x500.jpg': 0.47031423, 'BNedJec-Sq6qsMLHaXg09w_pl_480x640.png': 0.5006413, 'BNlEB7bzScKKmmTKqkYSxA_375x500.jpg': 0.48065907, 'BNygFMP4Sxal1ckUDR2yuQ_375x500.jpg': 0.48868445, 'bobal_almud.jpg': 0.53083545, 'BOfH4Sh6RdW9xjVXN2RTFQ_375x500.jpg': 0.5056777, 'bOfQdCLITMSZ9sJAid0Nrg_375x500.jpg': 0.5117592, 'BohOCDjDQuGRqIcm0xOE3Q_pl_480x640.png': 0.52837825, 'Boi1sMC8Ti2xGipVwBEKGw_pl_480x640.png': 0.5499019, 'bOkJJFNIQ0--4lLJ9cab4w_375x500.jpg': 0.36877483, 'bopqi61FT1ibni4qx18kYQ_375x500.jpg': 0.42603958, 'bowjrZT4S4Swasfe3vjxOA_pl_480x640.png': 0.59760886, 'bP4UgrQIRZGdGUg1q0p32w_375x500.jpg': 0.52663094, 'bpXmeTFhTdOEKL1S9T0MDw_375x500.jpg': 0.4847371, 'bpZft_WASQKzAptgtk3_tQ_375x500.jpg': 0.4942091, 'bpzXLj5dQNOzJBIPfw93uw_375x500.jpg': 0.49756628, 'BQhFLNuXQDqXBQ1KCOWFoA_375x500.jpg': 0.49608433, 'BQw9PwX0TUuanMDhEeXmbA_375x500.jpg': 0.45431918, 'bR5CVHTYRM696rwUCTnMyA_375x500.jpg': 0.45988283, 'bS6gl55iS7uPbGMQ7eTeFw_375x500.jpg': 0.5364934, 'bsbrvxhvQUKbYSHqA7tCPg_pl_480x640.png': 0.60446954, 'BsQd9vCcRuGIBAc38g6B5w_375x500.jpg': 0.5288265, 'BSZxKtNbThqXXyi2R1GJDw_375x500.jpg': 0.51948994, 'bt3erCgsT1SEmbUV-sHL7A_pl_480x640.png': 0.48252836, 'BtpD7cvcRJqXzNhOq7DBag_pl_480x640.png': 0.52621543, 'btQXKP4iTTOj4N37v1aEiA_375x500.jpg': 0.48747647, 'bU4F9IxGTiC8DBTBQGMnVw_375x500.jpg': 0.43306425, 'BUcoY1ecS3eaeLK3Ai-Fcw_375x500.jpg': 0.4744101, 'buIAfwRSTMSIJLsDUdUdHA_375x500.jpg': 0.49134806, 'BuLjhWcnRie0h2srg4nNTA_375x500.jpg': 0.5266412, 'BUTSslndTRefuiCS7oPDyA_375x500.jpg': 0.48251516, 'BUyfw5w9RXKscj05e6WgJw_pl_480x640.png': 0.47829646, 'BvlvD5OFSZWL3h4OGzVAZA_375x500.jpg': 0.48369867, 'BvM4Lg2gSDCu4nZeZzIq9Q_375x500.jpg': 0.4715674, 'bvNmn1xLT2qzYOweNAWC9Q_375x500.jpg': 0.497016, 'bvvm276sTXG3zNsI0hqHnA_pl_480x640.png': 0.5176913, 'BVwVX6GORLSArVQxjz1chw_pl_480x640.png': 0.54820305, 'BvZ_yxeKRIaObAKs31kfsw_375x500.jpg': 0.45233494, 'bwo-tY5cT9eDlzNdXiaTyQ_375x500.jpg': 0.5535873, 'BwpKv7ZbQry1123HOFvHkg_375x500.jpg': 0.41957462, 'BwYePVlJS863yeESl8OM0g_375x500.jpg': 0.47628418, 'bXJLyGnBT6KsunnKM2s2GQ_375x500.jpg': 0.5133565, 'BxnJ4xkvR56NfMsaUPwBIA_375x500.jpg': 0.5982765, 'BxrkoURfQNCmzE9KoKuFog_375x500.jpg': 0.47099894, 'byd48sfuTampCl3rWrAamQ_pl_480x640.png': 0.52866393, 'BYSH5lfSTrqrnFmbo9XKvw_375x500.jpg': 0.5098244, 'byVXjZe7SvSBn6s3L-X2Rg_pl_480x640.png': 0.50208706, 'Bz9wTLkMTM6VEORPVvWiqw_375x500.jpg': 0.46424988, 'bzFjWSb-TaqLg8ooJCrL4Q_375x500.jpg': 0.503309, 'BZlVIqKpQkiyiM6OL8P0cA_375x500.jpg': 0.47991875, 'bzwKgTYeTtOoLzHd661oAA_375x500.jpg': 0.5116065, 'B_dfR_zGTlevNlYFMvQYfg_375x500.jpg': 0.43858102, 'b_DYpEEqTWCxSJP2sgwK8Q_375x500.jpg': 0.53395444, 'C-2QWmJKT_uUn9_7s45mEg_375x500.jpg': 0.52537966, 'c0KU497dSXCBYysMOgcHFg_375x500.jpg': 0.5603681, 'c17IZ9Q2RhmnNmjv7cG24A_pl_480x640.png': 0.54497266, 'C1YLpe_eSOG7B09iqOE6nA_pl_480x640.png': 0.5952775, 'c2475-29SJiasOHDtujsOg_375x500.jpg': 0.46735072, 'c2L1r6amT76FeIHXmqGfEA_pl_480x640.png': 0.53152335, 'C2me62qDSNKxTuPwhhhAnA_pl_480x640.png': 0.5246961, 'C2_uHLVFS9Gq2BPpTbO8YQ_375x500.jpg': 0.47632182, 'C4Fta-ooQq-HAZlLGT82JA_375x500.jpg': 0.5622791, 'c4G239M6SemnZq5CxrHyMQ_pl_480x640.png': 0.5740724, 'c4K_vvKCRiCXAZ-U95kbpw_375x500.jpg': 0.45773432, 'c4MqmdMESSO6EUX7RyvmIw_pl_480x640.png': 0.48425463, 'c4sY-EKCT_a6NM-gRdxAQA_375x500.jpg': 0.4940975, 'C4Y_qrf_QAqruOi7GnKTvg_375x500.jpg': 0.4280108, 'c4zL6TeRSRuWAohbqASUCA_375x500.jpg': 0.49755144, 'c5bhXX1gSR-On8mcQXgHgQ_375x500.jpg': 0.506983, 'C5lPTDcJTBiZym_66wigeQ_375x500.jpg': 0.5070604, 'c5LPuPSATee7yxrxhwmdrA_375x500.jpg': 0.46720454, 'c5TTWz_eSVeogVcjmS0Iug_pl_480x640.png': 0.5418057, 'c67Q4VLjSne6EL0DH970eg_375x500.jpg': 0.54799247, 'C6E4XybFTsyOnmfjixTVzw_375x500.jpg': 0.5521718, 'C6MZNwG3Q4ulvxVdtGjoNA_375x500.jpg': 0.5112183, 'C7LOYGPURO-SLaxTF8lqpw_375x500.jpg': 0.44025883, 'c7V2heChQlqaHqKSiTV04g_375x500.jpg': 0.47894615, 'C88y9TUQRNy5hhmsFEOwFA_pl_480x640.png': 0.5385397, 'C8eVUJ3gQzqKsGG1ONq4Zg_375x500.jpg': 0.47717938, 'C8m_VWtOTKyLrzaE0YHSbw_375x500.jpg': 0.4556628, 'C8_3fKS3Ro2P1-GlA723Hw_375x500.jpg': 0.5446976, 'C95AF9guS2Ke4PszvC3XIQ_375x500.jpg': 0.46902674, 'Cau_k1_PROqxLCi3dWr7hA_375x500.jpg': 0.51385766, 'cave_jolimont.jpg': 0.5163976, 'cay7gYx7QlGb6JmEE9zN6w_375x500.jpg': 0.50447476, 'caYSSFg6QI6DqKS7t3RUUQ_375x500.jpg': 0.5631627, 'CB68PE1RSI-LkqhydjTcVg_375x500.jpg': 0.47851226, 'cCZpjb73SuK9rHV-X2nqdQ_375x500.jpg': 0.47835016, 'cE37_VDdReywKlmT5x3ehg_375x500.jpg': 0.4515605, 'cEmyYWkiQyqsx5qqE_R95g_375x500.jpg': 0.5617054, 'ceZ0eut8Se6Zu6SG8oiMlA_375x500.jpg': 0.4272344, 'cfobRXO0SB63dJGdSXleGQ_375x500.jpg': 0.53097683, 'CgdRLuzhT9uWktx5VU8f9Q_375x500.jpg': 0.4767294, 'cGpHIrHbR0i2nlUvEbG0Wg_375x500.jpg': 0.48186016, 'CH9vbD2RQ0WhQofLIvg1lA_375x500.jpg': 0.45788375, 'ChcMxA8oQkOIk8r-jMV_5w_375x500.jpg': 0.44743475, 'CHNhF6rBRa2VE-Ak_OJxQQ_375x500.jpg': 0.5201517, 'CiE90I0bSqm1CB1fMRTY-g_375x500.jpg': 0.54341054, 'cijmxH10RjSijlZ-U2Icdw_375x500.jpg': 0.50644034, 'Cjbru11GR8a8lawVL9hYmA_375x500.jpg': 0.51017195, 'CJp0mBcuQY-7zVfgw---vg_375x500.jpg': 0.5013188, 'CJVw7b_BQTq08gqYuv9ukw_375x500.jpg': 0.49798265, 'cK3XYYn3Tq-woJuPx_fgYw_375x500.jpg': 0.50066227, 'CKczsG54T96M8Ba0XAF0AA_375x500.jpg': 0.5082746, 'ckd-aoUuTZWkmzK7ugA_Ag_pl_480x640.png': 0.50484574, 'Ckgp6S2TToyzaZMpPhtyAw_375x500.jpg': 0.572387, 'CKwVtupgQmK6b027r0m_jw_375x500.jpg': 0.46782842, 'Cl0PKT_-Ru269Qdyl3qzgA_375x500.jpg': 0.50724953, 'cl86DBNERyuMJcdWmxX--g_375x500.jpg': 0.4258101, 'cldf2mtLTjSUKPcG_6_00Q_375x500.jpg': 0.5611209, 'CljCG9bRTGe8ubeBNeSiBw_pl_480x640.png': 0.5914013, 'clJQG5JHT2SxY3BnchdnkQ_375x500.jpg': 0.558797, 'CLpCUMaHSH2JGqey-EcR9w_375x500.jpg': 0.53800493, 'cMf6CWUHSLWaMW62xVsSSA_pl_480x640.png': 0.4818732, 'cNz6lzHNTHa8sSXYFS-SqQ_375x500.jpg': 0.4643439, 'Co7aYeWYQi6XuOm40U80BQ_375x500.jpg': 0.43770325, 'corton.jpg': 0.4674351, 'guillaume.jpg': 0.48928303, 'H&M_Hoffer.png': 0.5193309, 'lacroix.jpg': 0.59490883, 'pelican.jpg': 0.6128692}

In [29]:
type(comp['alsace_dopf.jpg'])

float

In [10]:
match_list = []
for k, v in comp.items():
    if v > 0.6:
        match_list.append(k)
        match_list.append(v)

In [11]:
match_list

['7FfnrtKqSs253irxE7msig_375x500.jpg',
 0.6260933,
 '7ImKFBtRTXW3O1BKzP2YOg_pl_480x640.png',
 0.61934143,
 '9968FEzCQiK_ohpYsgAtwg_pl_480x640.png',
 0.6152123,
 'A9wXtVLdR-y-BkhKkOjtUQ_pl_480x640.png',
 0.62825763,
 'AlytfdydQD27i6DNNpylxg_pl_480x640.png',
 0.6303919,
 'bI4ObnT4TIehDopl6tlGWw_pl_480x640.png',
 0.6410688,
 'bmCw1pm_RYOKt9ipk_A6Ig_pl_480x640.png',
 0.6005985,
 'bsbrvxhvQUKbYSHqA7tCPg_pl_480x640.png',
 0.60446954,
 'pelican.jpg',
 0.6128692]

In [14]:
td = {}
for i in range(len(match_list)):
    if type(match_list[i]) is str:
        td[match_list[i]] = match_list[i + 1]
print(td)       
    

{'7FfnrtKqSs253irxE7msig_375x500.jpg': 0.6260933, '7ImKFBtRTXW3O1BKzP2YOg_pl_480x640.png': 0.61934143, '9968FEzCQiK_ohpYsgAtwg_pl_480x640.png': 0.6152123, 'A9wXtVLdR-y-BkhKkOjtUQ_pl_480x640.png': 0.62825763, 'AlytfdydQD27i6DNNpylxg_pl_480x640.png': 0.6303919, 'bI4ObnT4TIehDopl6tlGWw_pl_480x640.png': 0.6410688, 'bmCw1pm_RYOKt9ipk_A6Ig_pl_480x640.png': 0.6005985, 'bsbrvxhvQUKbYSHqA7tCPg_pl_480x640.png': 0.60446954, 'pelican.jpg': 0.6128692}


In [4]:
counter = 0
for i in os.listdir('archive/directory'):
    counter += 1
print(counter)

996


In [3]:
path = os.path.abspath('archive/directory')
numb = 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
for i in os.listdir(path):
    p = Path(i)
    my_file = i
    new_ext = p.suffix
    name_without_ext = os.path.splitext(my_file)[0]
    os.rename(os.path.join(path, my_file), numb + new_ext)
    print(p.stem) # file name without extention
    print(p.suffix) # file extantion
#     p.rename(f"{numb}{p.suffix}")
    print(type(p))
    print(my_file)
    break

guillaume_test
.jpg


FileNotFoundError: [WinError 2] Не удается найти указанный файл: 'guillaume_test.jpg' -> 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA.jpg'

In [69]:
os.rename(os.path.join('archive/dataset_test', os.listdir('archive/dataset_test')[0]), numb)

In [ ]:
 my_file = 'E:\\seaborn_plot\\x.dwt'
 new_ext = '.txt'
 # Gets my_file minus the extension
 name_without_ext = os.path.splitext(my_file)[0]
 os.rename(my_file, name_without_ext + new_ext)

In [23]:
p = Path('archive/dataset_test')
# p.rename(Path(p.parent, f"{p.stem}_1_{p.suffix}"))

In [26]:
len(list(p.glob('**/*.png')))

254

In [35]:
p.suffix

''

In [76]:
os.path.abspath('archive/dataset_test')

'C:\\py_projects\\imgOVDoc\\archive\\dataset_test'

In [79]:
os.listdir(path)

['5FZUOH_XQkyIwjwlQ9rpjQ_375x500.jpg',
 '5gD25ppLQDGqJiSZG8Gt5A_375x500.jpg',
 '5gKGBBiFTHiyrZTHKKbpvg_375x500.jpg',
 '5Gn8SjTZQ9KlS6vNQAYFJw_375x500.jpg',
 '5gX4A5QbTQyg-980jbk3zw_pl_480x640.png',
 '5GZR3AxPRgiLtdK382X2rg_375x500.jpg',
 '5G_iTZkqRRiSx2qnxo7-GQ_375x500.jpg',
 '5HnI3RFvSSu7Hm9iOdGjnA_375x500.jpg',
 '5HqFVpycTDikvwB8qcSPxw_375x500.jpg',
 '5hz90xH8TZ-crlgitwPtMg_pl_480x640.png',
 '5Iflj6NISZa9SvNJyOmU4g_375x500.jpg',
 '5IphOlAtRGGxc9goQ4P-uQ_375x500.jpg',
 '5iqUzVDRR4aPEBV-AizQRQ_375x500.jpg',
 '5ITt9UhGRkCOt_NWirgEvw_375x500.jpg',
 '5IVe34gyS9GUg8wEAeglCg_375x500.jpg',
 '5iVKfLZZTzigaR-Hx-VFlA_375x500.jpg',
 '5Iz-jDR8Rtm_YBn0O2ixag_375x500.jpg',
 '5jHuQcKKTV2h3xoXDvMtFA_375x500.jpg',
 '5jP0Gw-aSK6N_4Eo5p-Z9g_375x500.jpg',
 '5jUji87xQ9qTjVbHjmdzVQ_375x500.jpg',
 '5jvQoGoOTwKeZIQmsf36sw_375x500.jpg',
 '5JZGJUJeRE-pnRMV1KvKtQ_375x500.jpg',
 '5KEdXLY9T9W227PTvEfFBw_375x500.jpg',
 '5KgnlX6uQd6Trcmi91DEAA_375x500.jpg',
 '5KihAzFgSN2-ejvXmyucSw_375x500.jpg',
 '5KjqVwh1QUayMaLJm